# TCC

# Etapa 1: Coleta e Validação dos Dados Brutos

## 1.1. Introdução



O ponto de partida de qualquer projeto de *data science* é a obtenção e validação dos dados. Esta etapa inicial é fundamental para garantir a integridade, a completude e a consistência estrutural dos dados brutos que servirão de alicerce para todas as fases subsequentes de análise e modelagem.

## 1.2. Objetivos





Os objetivos desta etapa são:
1.  **Estabelecer os caminhos** para os diretórios de entrada (dados brutos) e saída (logs e artefatos da Etapa 1).
2.  **Validar a presença** de todos os arquivos anuais esperados (2006 a 2023) no formato `.xlsx`.
3.  **Verificar a estrutura interna** de cada arquivo, assegurando que todas as planilhas mensais (`JAN`, `FEV`, ..., `DEZ`) existem.
4.  **Garantir a consistência do esquema**, confirmando que todas as planilhas, em todos os arquivos, possuem o mesmo conjunto de colunas.
5.  **Gerar um log detalhado** de todo o processo de validação, registrando sucessos, avisos e erros para futura referência.

## 1.3. Metodologia



A validação será executada por meio de um conjunto de funções programáticas que irão inspecionar sistematicamente os arquivos e suas estruturas. O resultado de cada verificação será impresso no console e armazenado em um arquivo de log, localizado em um diretório específico para esta etapa, garantindo que o processo seja auditável e reprodutível.

In [ ]:
import os
import pandas as pd
from datetime import datetime

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

RAW_DATA_DIR = os.path.join(DRIVE_BASE_PATH, 'xlsx_tratados')

STAGE_01_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_01_validation')

LOG_FILE_PATH = os.path.join(STAGE_01_OUTPUT_DIR, 'log_etapa_1.txt')

os.makedirs(STAGE_01_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE VALIDAÇÃO DA ETAPA 1 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*40 + "\n\n")

print(f"Diretório de saída da Etapa 1 criado em: {STAGE_01_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

Diretório de saída da Etapa 1 criado em: /content/drive/MyDrive/dados_tcc/stage_01_validation
Arquivo de log inicializado em: /content/drive/MyDrive/dados_tcc/stage_01_validation/log_etapa_1.txt


## 1.4. Funções de Validação



Para manter o código organizado e reutilizável, a lógica de validação foi encapsulada em funções específicas. Cada função tem uma responsabilidade única, facilitando a manutenção e a depuração.

-   **`log_mensagem`**: Uma função auxiliar para registrar mensagens tanto no console quanto no arquivo de log.
-   **`validar_anos`**: Verifica a existência dos arquivos `.xlsx` para cada ano esperado.
-   **`validar_meses_em_sheets`**: Confere se cada arquivo anual contém as 12 planilhas mensais.
-   **`validar_esquema_colunas`**: Compara as colunas de todas as planilhas com um esquema de referência para garantir a consistência.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """
    Registra uma mensagem formatada no console e no arquivo de log.

    Args:
        mensagem (str): A mensagem a ser registrada.
        tipo (str): O tipo de mensagem (ex: 'INFO', 'AVISO', 'ERRO').
    """
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a') as f:
        f.write(log_formatado + "\n")

def validar_anos(diretorio, anos_esperados):
    """
    Valida se os arquivos XLSX para os anos esperados existem no diretório.

    Args:
        diretorio (str): Caminho para a pasta com os arquivos XLSX.
        anos_esperados (list): Lista de inteiros representando os anos.

    Returns:
        tuple: Uma tupla contendo (lista de anos encontrados, lista de anos faltantes).
    """
    log_mensagem("Iniciando validação de anos existentes...")
    try:
        anos_existentes = [int(nome.split('.')[0]) for nome in os.listdir(diretorio) if nome.endswith('.xlsx')]
        anos_faltantes = [ano for ano in anos_esperados if ano not in anos_existentes]

        if not anos_faltantes:
            log_mensagem("Todos os anos esperados foram encontrados.")
        else:
            log_mensagem(f"Anos faltantes: {anos_faltantes}", tipo='AVISO')

        log_mensagem(f"Anos encontrados: {sorted(anos_existentes)}")
        return sorted(anos_existentes), anos_faltantes
    except Exception as e:
        log_mensagem(f"Falha ao validar anos: {e}", tipo='ERRO')
        return [], anos_esperados

def validar_meses_em_sheets(diretorio, anos_a_verificar):
    """
    Verifica se cada arquivo anual contém as 12 planilhas mensais.

    Args:
        diretorio (str): Caminho para a pasta com os arquivos XLSX.
        anos_a_verificar (list): Lista de anos cujos arquivos serão verificados.

    Returns:
        dict: Um dicionário com anos como chave e listas de sheets faltantes como valor.
    """
    log_mensagem("\nIniciando validação de sheets mensais...")
    meses_esperados = ['JAN', 'FEV', 'MAR', 'ABR', 'MAI', 'JUN', 'JUL', 'AGO', 'SET', 'OUT', 'NOV', 'DEZ']
    sheets_com_problemas = {}

    for ano in anos_a_verificar:
        caminho_arquivo = os.path.join(diretorio, f"{ano}.xlsx")
        try:
            xls = pd.ExcelFile(caminho_arquivo)
            nomes_sheets = xls.sheet_names

            sheets_faltantes = [f"{mes}-{ano}" for mes in meses_esperados if f"{mes}-{ano}" not in nomes_sheets]

            if sheets_faltantes:
                log_mensagem(f"No arquivo '{ano}.xlsx', sheets faltantes: {sheets_faltantes}", tipo='AVISO')
                sheets_com_problemas[ano] = sheets_faltantes
            else:
                log_mensagem(f"Arquivo '{ano}.xlsx' está completo com as 12 sheets mensais.")

        except Exception as e:
            log_mensagem(f"Não foi possível ler o arquivo '{ano}.xlsx': {e}", tipo='ERRO')
            sheets_com_problemas[ano] = ['ERRO DE LEITURA']

    if not sheets_com_problemas:
        log_mensagem("Todos os arquivos verificados possuem as sheets mensais corretas.")

    return sheets_com_problemas

def validar_esquema_colunas(diretorio, anos_a_verificar):
    """
    Verifica se todas as planilhas em todos os arquivos têm as mesmas colunas.

    Args:
        diretorio (str): Caminho para a pasta com os arquivos XLSX.
        anos_a_verificar (list): Lista de anos cujos arquivos serão verificados.

    Returns:
        dict: Dicionário com planilhas que apresentam colunas divergentes.
    """
    log_mensagem("\nIniciando validação de esquema de colunas...")
    if not anos_a_verificar:
        log_mensagem("Nenhum ano para verificar. Pulando validação de colunas.", tipo='AVISO')
        return {}

    try:
        primeiro_ano = anos_a_verificar[0]
        caminho_primeiro_arquivo = os.path.join(diretorio, f"{primeiro_ano}.xlsx")
        xls_primeiro = pd.ExcelFile(caminho_primeiro_arquivo)
        df_referencia = pd.read_excel(caminho_primeiro_arquivo, sheet_name=xls_primeiro.sheet_names[0])
        colunas_padrao = set(df_referencia.columns)
        log_mensagem(f"Esquema de referência definido com {len(colunas_padrao)} colunas a partir de '{primeiro_ano}.xlsx'.")
    except Exception as e:
        log_mensagem(f"Não foi possível definir o esquema de referência: {e}", tipo='ERRO')
        return {'ERRO_REFERENCIA': str(e)}

    colunas_com_problemas = {}
    for ano in anos_a_verificar:
        caminho_arquivo = os.path.join(diretorio, f"{ano}.xlsx")
        try:
            xls = pd.ExcelFile(caminho_arquivo)
            for sheet in xls.sheet_names:
                df_atual = pd.read_excel(caminho_arquivo, sheet_name=sheet)
                colunas_atuais = set(df_atual.columns)

                if colunas_atuais != colunas_padrao:
                    diferenca = colunas_atuais.symmetric_difference(colunas_padrao)
                    log_mensagem(f"Arquivo '{ano}.xlsx', sheet '{sheet}' possui colunas divergentes: {diferenca}", tipo='AVISO')
                    colunas_com_problemas[f"{ano}-{sheet}"] = list(diferenca)
        except Exception as e:
            log_mensagem(f"Erro ao processar a sheet '{sheet}' do arquivo '{ano}.xlsx': {e}", tipo='ERRO')
            colunas_com_problemas[f"{ano}-{sheet}"] = ['ERRO DE LEITURA']

    if not colunas_com_problemas:
        log_mensagem("Todas as planilhas verificadas possuem o mesmo esquema de colunas.")

    return colunas_com_problemas

## 1.5. Execução da Pipeline de Validação



A célula a seguir orquestra a execução das funções de validação. Ela define os parâmetros necessários (como o intervalo de anos esperado) e chama cada função na sequência correta. Ao final, um resumo do processo de validação é gerado no arquivo de log, concluindo formalmente a Etapa 1.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE VALIDAÇÃO DA ETAPA 1 ---")

ANOS_ESPERADOS = list(range(2006, 2024))
log_mensagem(f"Anos esperados para o projeto: de {ANOS_ESPERADOS[0]} a {ANOS_ESPERADOS[-1]}")

anos_encontrados, anos_faltantes = validar_anos(RAW_DATA_DIR, ANOS_ESPERADOS)

if not anos_encontrados:
    log_mensagem("Nenhum arquivo de dados encontrado. A validação não pode continuar.", tipo='ERRO CRÍTICO')
else:
    sheets_faltantes_dict = validar_meses_em_sheets(RAW_DATA_DIR, anos_encontrados)

    colunas_divergentes_dict = validar_esquema_colunas(RAW_DATA_DIR, anos_encontrados)

    log_mensagem("\n--- RESUMO DA VALIDAÇÃO DA ETAPA 1 ---", tipo='SUMÁRIO')
    log_mensagem(f"Total de anos faltantes: {len(anos_faltantes)}", tipo='SUMÁRIO')
    log_mensagem(f"Total de arquivos com sheets mensais faltantes: {len(sheets_faltantes_dict)}", tipo='SUMÁRIO')
    log_mensagem(f"Total de planilhas com esquema de colunas divergente: {len(colunas_divergentes_dict)}", tipo='SUMÁRIO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE VALIDAÇÃO DA ETAPA 1 ---")

[INFO] 20:44:19 - 
--- INICIANDO PIPELINE DE VALIDAÇÃO DA ETAPA 1 ---
[INFO] 20:44:19 - Anos esperados para o projeto: de 2006 a 2023
[INFO] 20:44:19 - Iniciando validação de anos existentes...
[INFO] 20:44:20 - Todos os anos esperados foram encontrados.
[INFO] 20:44:20 - Anos encontrados: [2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]
[INFO] 20:44:20 - 
Iniciando validação de sheets mensais...
[INFO] 20:44:24 - Arquivo '2006.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:28 - Arquivo '2007.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:32 - Arquivo '2008.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:36 - Arquivo '2009.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:40 - Arquivo '2010.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:43 - Arquivo '2011.xlsx' está completo com as 12 sheets mensais.
[INFO] 20:44:47 - Arquivo '2012.xlsx' está completo com as 12 shee

# Etapa 2: Preparação da Tabela de Consulta Geográfica

## 2.1. Introdução




Após validar a integridade estrutural dos dados na Etapa 1, o próximo passo é enriquecê-los. A localização de um imóvel é um dos fatores mais preponderantes na determinação de seu valor. No entanto, os dados brutos contêm apenas o Código de Endereçamento Postal (CEP), uma informação que, por si só, possui baixo valor preditivo direto.

Esta etapa foca na criação de um artefato de dados fundamental: uma tabela de consulta que traduz cada CEP único encontrado nos dados transacionais em informações geográficas de alto valor, como bairro e coordenadas (latitude e longitude).


## 2.2. Objetivos




1.  **Extrair e Unificar CEPs:** Ler todos os arquivos de dados brutos para extrair e consolidar uma lista completa de todos os CEPs mencionados.
2.  **Mapear com Dados Geográficos:** Cruzar a lista de CEPs únicos com um arquivo mestre de geolocalização para obter os atributos geográficos correspondentes.
3.  **Criar a Tabela de Consulta:** Gerar um arquivo `.csv` limpo e otimizado que servirá como a principal fonte de dados geográficos para a Etapa 3. Este arquivo conterá a relação `CEP -> Bairro, Longitude, Latitude`.
4.  **Identificar CEPs Faltantes:** Produzir um relatório de CEPs que existem nos dados transacionais mas não foram encontrados no arquivo mestre, possibilitando uma análise futura da qualidade dos dados.
5.  **Manter Rastreabilidade:** Registrar todo o processo em um arquivo de log detalhado.


## 2.3. Metodologia



O processo consiste em duas fases principais. Primeiro, uma varredura completa nos arquivos `.xlsx` é realizada para extrair todos os CEPs. Segundo, esses CEPs são comparados a uma base de dados geográfica externa (`sp_data.xlsx`). O resultado desta comparação é materializado em dois arquivos de saída: a tabela de consulta com os CEPs mapeados e uma lista com os CEPs que não puderam ser encontrados, garantindo que nenhuma informação seja perdida.

In [ ]:
import os
import pandas as pd
from datetime import datetime

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

RAW_DATA_DIR = os.path.join(DRIVE_BASE_PATH, 'xlsx_tratados')
MASTER_GEO_FILE_PATH = os.path.join(DRIVE_BASE_PATH, 'dados_cep', 'sp_data.xlsx')

STAGE_02_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_02_geo_lookup')

LOG_FILE_PATH = os.path.join(STAGE_02_OUTPUT_DIR, 'log_etapa_2.txt')
GEO_LOOKUP_TABLE_PATH = os.path.join(STAGE_02_OUTPUT_DIR, 'ceps_geolocalizados.csv')
MISSING_CEPS_PATH = os.path.join(STAGE_02_OUTPUT_DIR, 'ceps_faltantes_no_mestre.csv')

os.makedirs(STAGE_02_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE PREPARAÇÃO GEOGRÁFICA DA ETAPA 2 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*50 + "\n\n")

print(f"Diretório de saída da Etapa 2 criado em: {STAGE_02_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

Diretório de saída da Etapa 2 criado em: /content/drive/MyDrive/dados_tcc/stage_02_geo_lookup
Arquivo de log inicializado em: /content/drive/MyDrive/dados_tcc/stage_02_geo_lookup/log_etapa_2.txt


## 2.4. Funções de Processamento



A lógica desta etapa está contida nas seguintes funções, que garantem um fluxo de trabalho modular e claro.

-   **`log_mensagem`**: A mesma função auxiliar da etapa anterior para registro de logs.
-   **`extrair_ceps_de_todos_arquivos`**: Itera sobre todos os arquivos `.xlsx` e extrai a coluna de CEP de cada planilha.
-   **`criar_tabela_consulta_geo`**: Pega a lista de CEPs extraídos, compara com o arquivo mestre e gera os DataFrames de saída.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """
    Registra uma mensagem formatada no console e no arquivo de log.
    """
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a') as f:
        f.write(log_formatado + "\n")

def extrair_ceps_de_todos_arquivos(diretorio):
    """
    Extrai todos os CEPs não nulos de todos os arquivos XLSX no diretório.

    Args:
        diretorio (str): Caminho para a pasta com os arquivos XLSX.

    Returns:
        pd.DataFrame: Um DataFrame com uma única coluna 'CEP' contendo todos os CEPs extraídos.
    """
    log_mensagem("Iniciando extração de CEPs de todos os arquivos brutos...")
    lista_de_ceps = []

    arquivos_xlsx = [f for f in os.listdir(diretorio) if f.endswith('.xlsx')]
    log_mensagem(f"Encontrados {len(arquivos_xlsx)} arquivos .xlsx para processar.")

    for arquivo in arquivos_xlsx:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        try:
            xls = pd.ExcelFile(caminho_arquivo)
            for sheet in xls.sheet_names:
                df = pd.read_excel(caminho_arquivo, sheet_name=sheet)
                if 'CEP' in df.columns:
                    ceps_validos = pd.to_numeric(df['CEP'], errors='coerce').dropna().astype(int)
                    lista_de_ceps.extend(ceps_validos.tolist())
            log_mensagem(f"Processado com sucesso o arquivo: {arquivo}")
        except Exception as e:
            log_mensagem(f"Erro ao processar o arquivo {arquivo}: {e}", tipo='ERRO')

    log_mensagem(f"Extração finalizada. Total de {len(lista_de_ceps)} registros de CEP encontrados (com duplicatas).")
    return pd.DataFrame(lista_de_ceps, columns=['CEP'])

def criar_tabela_consulta_geo(df_ceps_extraidos, caminho_geo_mestre):
    """
    Compara os CEPs extraídos com o arquivo mestre e cria a tabela de consulta.

    Args:
        df_ceps_extraidos (pd.DataFrame): DataFrame com os CEPs dos arquivos brutos.
        caminho_geo_mestre (str): Caminho para o arquivo sp_data.xlsx.

    Returns:
        tuple: Uma tupla contendo (df_lookup_table, df_ceps_faltantes) ou (None, None) em caso de erro.
    """
    log_mensagem("Iniciando criação da tabela de consulta geográfica...")

    try:
        ceps_unicos_extraidos = set(df_ceps_extraidos['CEP'].unique())
        log_mensagem(f"Total de CEPs únicos a serem mapeados: {len(ceps_unicos_extraidos)}")

        df_geo_mestre = pd.read_excel(caminho_geo_mestre)
        df_geo_mestre['CEP'] = pd.to_numeric(df_geo_mestre['CEP'], errors='coerce').dropna().astype(int)
        ceps_unicos_mestre = set(df_geo_mestre['CEP'].unique())
        log_mensagem(f"Arquivo mestre de geolocalização carregado com {len(ceps_unicos_mestre)} CEPs únicos.")

        ceps_encontrados = ceps_unicos_extraidos.intersection(ceps_unicos_mestre)
        ceps_faltantes = ceps_unicos_extraidos.difference(ceps_unicos_mestre)

        log_mensagem(f"CEPs encontrados no arquivo mestre: {len(ceps_encontrados)}")
        log_mensagem(f"CEPs não encontrados no arquivo mestre: {len(ceps_faltantes)}", tipo='AVISO' if ceps_faltantes else 'INFO')

        df_lookup_table = df_geo_mestre[df_geo_mestre['CEP'].isin(ceps_encontrados)].drop_duplicates(subset=['CEP']).reset_index(drop=True)
        log_mensagem(f"Tabela de consulta criada com {len(df_lookup_table)} entradas únicas.")

        df_ceps_faltantes = pd.DataFrame(list(ceps_faltantes), columns=['CEP'])

        return df_lookup_table, df_ceps_faltantes

    except FileNotFoundError:
        log_mensagem(f"Arquivo mestre de geolocalização não encontrado em: {caminho_geo_mestre}", tipo='ERRO CRÍTICO')
        return None, None
    except Exception as e:
        log_mensagem(f"Ocorreu um erro inesperado ao criar a tabela de consulta: {e}", tipo='ERRO CRÍTICO')
        return None, None

## 2.5. Execução da Pipeline de Preparação Geográfica



A célula final orquestra a execução das funções. Ela chama a função de extração, passa os resultados para a função de criação da tabela de consulta e, por fim, salva os artefatos gerados (`.csv`) no diretório de saída da etapa. O log é atualizado em cada passo, garantindo a rastreabilidade completa do processo.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE PREPARAÇÃO GEOGRÁFICA DA ETAPA 2 ---")

df_ceps_extraidos = extrair_ceps_de_todos_arquivos(RAW_DATA_DIR)

if df_ceps_extraidos.empty:
    log_mensagem("Nenhum CEP foi extraído. A etapa não pode continuar.", tipo='ERRO CRÍTICO')
else:
    df_tabela_final, df_faltantes = criar_tabela_consulta_geo(df_ceps_extraidos, MASTER_GEO_FILE_PATH)

    if df_tabela_final is not None:
        try:
            df_tabela_final.to_csv(GEO_LOOKUP_TABLE_PATH, index=False, encoding='utf-8')
            log_mensagem(f"Tabela de consulta geográfica salva em: {GEO_LOOKUP_TABLE_PATH}")

            if not df_faltantes.empty:
                df_faltantes.to_csv(MISSING_CEPS_PATH, index=False, encoding='utf-8')
                log_mensagem(f"Arquivo com CEPs não encontrados salvo em: {MISSING_CEPS_PATH}", tipo='AVISO')
            else:
                log_mensagem("Não houve CEPs faltantes para salvar.")

        except Exception as e:
            log_mensagem(f"Erro ao salvar os arquivos de saída: {e}", tipo='ERRO')

log_mensagem("\n--- RESUMO DA PREPARAÇÃO GEOGRÁFICA DA ETAPA 2 ---", tipo='SUMÁRIO')
if 'df_tabela_final' in locals() and df_tabela_final is not None:
    log_mensagem(f"Total de CEPs únicos mapeados e salvos: {len(df_tabela_final)}", tipo='SUMÁRIO')
    log_mensagem(f"Total de CEPs não encontrados no mestre: {len(df_faltantes)}", tipo='SUMÁRIO')
    log_mensagem(f"Artefatos salvos em: {STAGE_02_OUTPUT_DIR}", tipo='SUMÁRIO')
else:
    log_mensagem("A etapa foi concluída com erros e nenhum artefato foi gerado.", tipo='SUMÁRIO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE PREPARAÇÃO GEOGRÁFICA DA ETAPA 2 ---")

[INFO] 21:02:07 - 
--- INICIANDO PIPELINE DE PREPARAÇÃO GEOGRÁFICA DA ETAPA 2 ---
[INFO] 21:02:07 - Iniciando extração de CEPs de todos os arquivos brutos...
[INFO] 21:02:07 - Encontrados 19 arquivos .xlsx para processar.
[INFO] 21:02:46 - Processado com sucesso o arquivo: 2007.xlsx
[INFO] 21:03:27 - Processado com sucesso o arquivo: 2010.xlsx
[INFO] 21:04:06 - Processado com sucesso o arquivo: 2012.xlsx
[INFO] 21:04:36 - Processado com sucesso o arquivo: 2015.xlsx
[INFO] 21:05:03 - Processado com sucesso o arquivo: 2016.xlsx
[INFO] 21:05:32 - Processado com sucesso o arquivo: 2017.xlsx
[INFO] 21:06:04 - Processado com sucesso o arquivo: 2018.xlsx
[INFO] 21:06:47 - Processado com sucesso o arquivo: 2008.xlsx
[INFO] 21:07:27 - Processado com sucesso o arquivo: 2009.xlsx
[INFO] 21:08:08 - Processado com sucesso o arquivo: 2013.xlsx
[INFO] 21:08:44 - Processado com sucesso o arquivo: 2014.xlsx
[INFO] 21:09:21 - Processado com sucesso o arquivo: 2006.xlsx
[INFO] 21:10:02 - Processado com s

# Etapa 3: Construção e Limpeza do Dataset Principal

## 3.1. Introdução




Com os dados brutos validados (Etapa 1) e a tabela de consulta geográfica preparada (Etapa 2), esta etapa foca em construir o dataset principal do projeto. O objetivo é transformar a coleção de arquivos anuais `.xlsx` em um único DataFrame, estruturado, limpo e enriquecido.

Este processo é o coração da preparação de dados, pois unifica informações de múltiplas fontes (arquivos anuais e a tabela geográfica) em uma única tabela coesa, que servirá como base para toda a engenharia de features e modelagem subsequentes.


## 3.2. Objetivos




1.  **Consolidar Dados Transacionais:** Ler todos os arquivos e planilhas de dados brutos e uni-los em um único DataFrame.
2.  **Padronizar o Esquema:** Aplicar um mapeamento de colunas consistente para padronizar os nomes e facilitar a manipulação dos dados.
3.  **Realizar Limpeza Primária:** Converter os tipos de dados para formatos adequados (numérico, data), remover duplicatas e aplicar filtros de domínio para reter apenas os imóveis de interesse (residenciais).
4.  **Enriquecer com Dados Geográficos:** Utilizar a tabela de consulta criada na Etapa 2 para adicionar informações de Bairro, Longitude e Latitude a cada transação, através de uma junção pela coluna `CEP`.
5.  **Armazenar o Dataset Consolidado:** Salvar o DataFrame final em um formato eficiente (Parquet) para otimizar o armazenamento e a velocidade de leitura nas próximas etapas.


## 3.3. Metodologia



A metodologia desta etapa segue um fluxo de trabalho sequencial:
1.  **Leitura e Concatenação:** Todos os dados são lidos em memória e empilhados.
2.  **Limpeza e Filtragem:** São aplicadas transformações para garantir a qualidade e a relevância dos dados.
3.  **Junção (Merge):** Os dados transacionais são cruzados com os dados geográficos.
4.  **Salvamento:** O produto final é persistido em disco.

Cada passo é meticulosamente registrado em log para garantir total transparência e reprodutibilidade.

In [ ]:
import os
import pandas as pd
from datetime import datetime

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

RAW_DATA_DIR = os.path.join(DRIVE_BASE_PATH, 'xlsx_tratados')
GEO_LOOKUP_TABLE_PATH = os.path.join(DRIVE_BASE_PATH, 'stage_02_geo_lookup', 'ceps_geolocalizados.csv')

STAGE_03_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_03_consolidated_data')

LOG_FILE_PATH = os.path.join(STAGE_03_OUTPUT_DIR, 'log_etapa_3.txt')
CONSOLIDATED_DATASET_PATH = os.path.join(STAGE_03_OUTPUT_DIR, 'dataset_consolidado.parquet')

os.makedirs(STAGE_03_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE CONSTRUÇÃO DO DATASET PRINCIPAL DA ETAPA 3 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 3 criado em: {STAGE_03_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

Diretório de saída da Etapa 3 criado em: /content/drive/MyDrive/dados_tcc/stage_03_consolidated_data
Arquivo de log inicializado em: /content/drive/MyDrive/dados_tcc/stage_03_consolidated_data/log_etapa_3.txt


## 3.4. Mapeamento de Colunas e Funções de Processamento



Para garantir um esquema de dados padronizado e legível, definimos um dicionário de mapeamento que traduz os nomes originais das colunas para um formato `snake_case`. As funções abaixo encapsulam a lógica para consolidação, limpeza e enriquecimento do dataset.

In [ ]:
MAPEAMENTO_COLUNAS = {
    'N° do Cadastro (SQL)': 'n_cadastro',
    'Nome do Logradouro': 'nome_logradouro',
    'Número': 'numero',
    'Complemento': 'complemento',
    'Bairro': 'bairro_original',
    'Referência': 'referencia',
    'CEP': 'cep',
    'Natureza de Transação': 'natureza_transacao',
    'Valor de Transação (declarado pelo contribuinte)': 'valor_transacao',
    'Data de Transação': 'data_transacao',
    'Valor Venal de Referência': 'valor_venal_referencia',
    'Proporção Transmitida (%)': 'proporcao_transmitida',
    'Valor Venal de Referência (proporcional)': 'valor_venal_referencia_proporcional',
    'Base de Cálculo adotada': 'base_calculo_original',
    'Tipo de Financiamento': 'tipo_financiamento',
    'Valor Financiado': 'valor_financiado',
    'Cartório de Registro': 'cartorio_de_registro',
    'Matrícula do Imóvel': 'matricula_imovel',
    'Situação do SQL': 'situacao_no_sql',
    'Área do Terreno (m2)': 'area_terreno',
    'Testada (m)': 'testada',
    'Fração Ideal': 'fracao_ideal',
    'Área Construída (m2)': 'area_construida',
    'Uso (IPTU)': 'uso_iptu',
    'Descrição do uso (IPTU)': 'descricao_uso_iptu',
    'Padrão (IPTU)': 'padrao_iptu',
    'Descrição do padrão (IPTU)': 'descricao_padrao_iptu',
    'ACC (IPTU)': 'acc_iptu'
}

def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a') as f:
        f.write(log_formatado + "\n")

def consolidar_dados_brutos(diretorio, mapeamento):
    """Lê, renomeia e consolida todos os arquivos XLSX em um único DataFrame."""
    log_mensagem("Iniciando consolidação dos dados brutos...")
    lista_dfs = []
    for arquivo in [f for f in os.listdir(diretorio) if f.endswith('.xlsx')]:
        ano = int(arquivo.split('.')[0])
        caminho_arquivo = os.path.join(diretorio, arquivo)
        try:
            xls = pd.ExcelFile(caminho_arquivo)
            for sheet in xls.sheet_names:
                df_sheet = pd.read_excel(caminho_arquivo, sheet_name=sheet)
                df_sheet.columns = df_sheet.columns.str.strip()
                df_sheet = df_sheet.rename(columns=mapeamento)
                df_sheet['ANO'] = ano
                df_sheet['MES_ANO'] = f"{ano}-{sheet.split('-')[0]}"
                lista_dfs.append(df_sheet)
            log_mensagem(f"Arquivo '{arquivo}' consolidado com sucesso.")
        except Exception as e:
            log_mensagem(f"Erro ao processar o arquivo {arquivo}: {e}", tipo='ERRO')

    df_consolidado = pd.concat(lista_dfs, ignore_index=True)
    log_mensagem(f"Consolidação concluída. Total de {len(df_consolidado)} registros.")
    return df_consolidado

def limpar_e_filtrar_dataset(df):
    """Aplica a limpeza primária e os filtros de domínio no dataset."""
    log_mensagem("Iniciando limpeza primária e filtragem do dataset...")

    registros_antes = len(df)
    df = df.drop_duplicates().reset_index(drop=True)
    log_mensagem(f"{registros_antes - len(df)} linhas duplicadas foram removidas.")

    registros_antes = len(df)
    filtro_residencial = df['descricao_padrao_iptu'].str.contains(
        'RESIDENCIAL VERTICAL|RESIDENCIAL HORIZONTAL',
        case=False,
        na=False
    )
    df = df[filtro_residencial].copy()
    df.reset_index(drop=True, inplace=True)
    log_mensagem(f"{registros_antes - len(df)} registros não residenciais foram removidos.")

    df['data_transacao'] = pd.to_datetime(df['data_transacao'], format='%d/%m/%Y', errors='coerce')
    log_mensagem("Tipos de dados de colunas de data foram convertidos.")

    return df

def enriquecer_com_geo(df_principal, caminho_lookup):
    """Enriquece o dataset principal com os dados geográficos e remove colunas desnecessárias."""
    log_mensagem("Iniciando enriquecimento com dados geográficos...")

    log_mensagem(f"Colunas disponíveis para merge: {df_principal.columns.tolist()}", tipo='DEBUG')

    try:
        df_lookup = pd.read_csv(caminho_lookup)
        log_mensagem(f"Tabela de consulta geográfica carregada com {len(df_lookup)} registros.")
    except FileNotFoundError:
        log_mensagem(f"Arquivo de lookup não encontrado em '{caminho_lookup}'. Processo interrompido.", tipo='ERRO CRÍTICO')
        return None

    df_principal['cep'] = pd.to_numeric(df_principal['cep'], errors='coerce')
    df_principal = df_principal.dropna(subset=['cep'])
    df_principal['cep'] = df_principal['cep'].astype(int)

    registros_antes = len(df_principal)
    df_enriquecido = pd.merge(df_principal, df_lookup, on='cep', how='left')

    registros_mapeados = df_enriquecido['Bairro'].notna().sum()
    percentual_mapeado = (registros_mapeados / registros_antes) * 100 if registros_antes > 0 else 0
    log_mensagem(f"{registros_mapeados} de {registros_antes} registros ({percentual_mapeado:.2f}%) foram enriquecidos com dados geográficos.")

    registros_antes = len(df_enriquecido)
    df_enriquecido = df_enriquecido.dropna(subset=['Bairro']).reset_index(drop=True)
    log_mensagem(f"{registros_antes - len(df_enriquecido)} registros sem correspondência geográfica foram removidos.")

    colunas_para_remover = [
        'nome_logradouro',
        'numero',
        'complemento',
        'bairro_original',
        'referencia',
        'cartorio_de_registro',
        'situacao_no_sql'
    ]
    df_enriquecido = df_enriquecido.drop(columns=colunas_para_remover, errors='ignore')
    log_mensagem(f"Colunas desnecessárias removidas do dataset: {colunas_para_remover}")

    return df_enriquecido

## 3.5. Execução da Pipeline de Construção do Dataset



A célula a seguir orquestra a execução de todas as funções definidas. O processo sequencial garante que os dados brutos sejam primeiramente consolidados, depois limpos e filtrados, e finalmente enriquecidos. O resultado é um único arquivo Parquet, que representa o estado mais atual e completo dos dados até o momento.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE CONSTRUÇÃO DO DATASET DA ETAPA 3 ---")

df_consolidado = consolidar_dados_brutos(RAW_DATA_DIR, MAPEAMENTO_COLUNAS)

if not df_consolidado.empty:
    df_limpo = limpar_e_filtrar_dataset(df_consolidado)

    df_final = enriquecer_com_geo(df_limpo, GEO_LOOKUP_TABLE_PATH)

    if df_final is not None and not df_final.empty:
        try:
            df_final.to_parquet(CONSOLIDATED_DATASET_PATH)
            log_mensagem(f"Dataset consolidado e enriquecido salvo com sucesso em: {CONSOLIDATED_DATASET_PATH}")
        except Exception as e:
            log_mensagem(f"Falha ao salvar o arquivo Parquet: {e}", tipo='ERRO')
    else:
        log_mensagem("O DataFrame final está vazio ou nulo. Nenhum arquivo foi salvo.", tipo='AVISO')
else:
    log_mensagem("Nenhum dado foi consolidado. Processo interrompido.", tipo='ERRO CRÍTICO')

log_mensagem("\n--- RESUMO DA CONSTRUÇÃO DO DATASET DA ETAPA 3 ---", tipo='SUMÁRIO')
if 'df_final' in locals() and df_final is not None:
    log_mensagem(f"Total de registros no dataset final: {len(df_final)}", tipo='SUMÁRIO')
    log_mensagem(f"Total de colunas no dataset final: {len(df_final.columns)}", tipo='SUMÁRIO')
    log_mensagem(f"Intervalo de anos presente: de {df_final['ANO'].min()} a {df_final['ANO'].max()}", tipo='SUMÁRIO')
else:
    log_mensagem("A etapa foi concluída com erros e o dataset consolidado não foi gerado.", tipo='SUMÁRIO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE CONSTRUÇÃO DO DATASET DA ETAPA 3 ---")

# Etapa 4: Engenharia de Features

## 4.1. Introdução




A Engenharia de Features é o processo de utilizar o conhecimento de domínio para criar variáveis que auxiliam os algoritmos de machine learning a preverem melhor. O dataset consolidado na Etapa 3, embora limpo e estruturado, contém colunas que não representam diretamente o "valor real" de um imóvel em um determinado momento no tempo.

Nesta etapa, vamos transformar esses dados brutos. As ações mais importantes serão recalcular a base de cálculo do imposto, que servirá como nosso proxy para o valor do imóvel, e ajustar este valor pela inflação acumulada ao longo dos anos. Este valor corrigido se tornará a nossa **variável-alvo** (o que queremos prever). Adicionalmente, criaremos outras features relevantes, como o preço por metro quadrado.


## 4.2. Objetivos




1.  **Aplicar Tipos de Dados Corretos:** Garantir que todas as colunas numéricas utilizadas nos cálculos estejam no formato correto para evitar erros.
2.  **Recalcular a Base de Cálculo:** Implementar a regra de negócio que define a `base_calculo` como o valor máximo entre o valor da transação e o valor venal de referência proporcional.
3.  **Criar a Variável-Alvo:** Ajustar a `base_calculo` pela inflação, trazendo todos os valores para uma base monetária equivalente (2024), criando a coluna `base_calculo_corrigida_2024`.
4.  **Tratar Outliers:** Remover valores extremos da variável-alvo para aumentar a estabilidade e a performance do modelo.
5.  **Criar Features Derivadas:** Calcular novas variáveis, como o preço por metro quadrado construído e do terreno, com base no valor corrigido.
6.  **Finalizar o Dataset:** Remover colunas intermediárias que foram usadas para os cálculos, resultando em um dataset enxuto e pronto para a próxima etapa.


## 4.3. Metodologia



O processo será executado através de uma série de funções que transformam o DataFrame sequencialmente. Cada função encapsula uma lógica de negócio ou uma transformação estatística específica. O resultado é um novo arquivo Parquet, contendo o dataset com todas as features de engenharia, e um log detalhado de todas as transformações realizadas.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_03_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_03_consolidated_data')
CONSOLIDATED_DATASET_PATH = os.path.join(STAGE_03_OUTPUT_DIR, 'dataset_consolidado.parquet')

STAGE_04_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_04_feature_engineered')

LOG_FILE_PATH = os.path.join(STAGE_04_OUTPUT_DIR, 'log_etapa_4.txt')
FEATURE_ENGINEERED_DATASET_PATH = os.path.join(STAGE_04_OUTPUT_DIR, 'dataset_com_features.parquet')

os.makedirs(STAGE_04_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE ENGENHARIA DE FEATURES DA ETAPA 4 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*50 + "\n\n")

print(f"Diretório de saída da Etapa 4 criado em: {STAGE_04_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

## 4.4. Funções de Engenharia de Features



As funções a seguir implementam o pipeline de transformações desta etapa. Elas são projetadas para serem aplicadas em sequência, cada uma adicionando ou modificando features no dataset.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a') as f:
        f.write(log_formatado + "\n")

def aplicar_tipos_e_limpeza(df):
    """Garante que as colunas numéricas estejam no formato correto para cálculo."""
    log_mensagem("Iniciando conversão de tipos de dados numéricos...")
    colunas_numericas = [
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'valor_financiado',
        'area_terreno', 'testada', 'fracao_ideal', 'area_construida'
    ]
    for col in colunas_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')

    log_mensagem("Conversão de tipos concluída.")
    return df

def calcular_base_de_calculo(df):
    """Recalcula a base de cálculo com base nas regras de negócio."""
    log_mensagem("Iniciando recálculo da base de cálculo...")

    df['valor_venal_referencia_proporcional'] = np.where(
        (df['proporcao_transmitida'] > 0) & (df['proporcao_transmitida'] <= 1),
        df['valor_venal_referencia'],
        df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    )

    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)

    log_mensagem("Base de cálculo recalculada com sucesso.")
    return df

def corrigir_pela_inflacao(df):
    """Ajusta a base de cálculo pela inflação, criando a variável-alvo."""
    log_mensagem("Iniciando correção de valores pela inflação...")
    inflacao_anual = {
        2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91,
        2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06,
        2022: 5.79, 2023: 4.62, 2024: 4.83,
    }

    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0

    for ano in anos_ordenados:
        fatores_correcao[ano] = fator_acumulado
        fator_acumulado *= fatores_anual[ano]

    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)

    log_mensagem("Variável-alvo 'base_calculo_corrigida_2024' criada com sucesso.")
    log_mensagem(f"Estatísticas da variável-alvo (antes de tratar outliers): "
                 f"Média={df['base_calculo_corrigida_2024'].mean():.2f}, "
                 f"Mediana={df['base_calculo_corrigida_2024'].median():.2f}")

    return df

def remover_outliers(df, coluna, quantil_inferior=0.05, quantil_superior=0.90):
    """Remove outliers de uma coluna com base em quantis."""
    log_mensagem(f"Iniciando remoção de outliers da coluna '{coluna}'...")
    registros_antes = len(df)

    limite_inferior = df[coluna].quantile(quantil_inferior)
    limite_superior = df[coluna].quantile(quantil_superior)

    df_filtrado = df[(df[coluna] >= limite_inferior) & (df[coluna] <= limite_superior)].copy()

    registros_removidos = registros_antes - len(df_filtrado)
    log_mensagem(f"{registros_removidos} registros ({registros_removidos/registros_antes:.2%}) foram removidos como outliers.")
    log_mensagem(f"Dados mantidos entre R$ {limite_inferior:,.2f} e R$ {limite_superior:,.2f}.")

    return df_filtrado

def criar_features_de_preco(df):
    """Cria features de preço por metro quadrado."""
    log_mensagem("Criando features de preço por m²...")
    df['preco_m2_construido_corrigido_2024'] = (df['base_calculo_corrigida_2024'] / df['area_construida']).replace([np.inf, -np.inf], np.nan)
    df['preco_m2_terreno_corrigido_2024'] = (df['base_calculo_corrigida_2024'] / df['area_terreno']).replace([np.inf, -np.inf], np.nan)
    log_mensagem("Features de preço por m² criadas.")
    return df

def finalizar_dataset(df):
    """Remove colunas intermediárias e desnecessárias."""
    log_mensagem("Iniciando limpeza final do dataset...")
    colunas_para_remover = [
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original',
        'base_calculo', 'fator_correcao'
    ]
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df.columns]

    df_final = df.drop(columns=colunas_existentes_para_remover)
    log_mensagem(f"Colunas removidas: {colunas_existentes_para_remover}")
    return df_final

## 4.5. Execução da Pipeline de Engenharia de Features



A célula a seguir orquestra a aplicação de todas as funções de engenharia de features. O processo começa carregando o dataset consolidado da Etapa 3 e aplica cada transformação de forma sequencial, registrando o impacto de cada passo. O resultado é o dataset que será efetivamente usado para treinar e avaliar os modelos de machine learning.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE ENGENHARIA DE FEATURES DA ETAPA 4 ---")

try:
    df = pd.read_parquet(CONSOLIDATED_DATASET_PATH)
    log_mensagem(f"Dataset consolidado carregado com {len(df)} registros e {len(df.columns)} colunas.")

    df = aplicar_tipos_e_limpeza(df)
    df = calcular_base_de_calculo(df)
    df = corrigir_pela_inflacao(df)
    df = remover_outliers(df, coluna='base_calculo_corrigida_2024', quantil_inferior=0.05, quantil_superior=0.95)
    df = criar_features_de_preco(df)
    df_final = finalizar_dataset(df)

    df_final.to_parquet(FEATURE_ENGINEERED_DATASET_PATH)
    log_mensagem(f"Dataset com features de engenharia salvo com sucesso em: {FEATURE_ENGINEERED_DATASET_PATH}")

    log_mensagem("\n--- RESUMO DA ENGENHARIA DE FEATURES DA ETAPA 4 ---", tipo='SUMÁRIO')
    log_mensagem(f"Total de registros no dataset final: {len(df_final)}", tipo='SUMÁRIO')
    log_mensagem(f"Total de colunas no dataset final: {len(df_final.columns)}", tipo='SUMÁRIO')
    log_mensagem(f"Colunas do dataset final: {df_final.columns.tolist()}", tipo='SUMÁRIO')

except FileNotFoundError:
    log_mensagem(f"Arquivo de entrada não encontrado: {CONSOLIDATED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado durante a execução da Etapa 4: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE ENGENHARIA DE FEATURES DA ETAPA 4 ---")

# Etapa 5: Análise Exploratória de Dados (EDA)

## 5.1. Introdução





Com um dataset limpo, consolidado e enriquecido com features de engenharia, esta etapa é dedicada à Análise Exploratória de Dados (EDA). O objetivo da EDA é desenvolver uma compreensão profunda sobre o dataset, descobrindo padrões, identificando anomalias, testando hipóteses e verificando suposições através de estatísticas descritivas e visualizações gráficas.

Esta fase é fundamental para guiar a etapa de modelagem. Os insights aqui gerados — como a distribuição da nossa variável-alvo, a relação entre as features e as tendências temporais — informarão a seleção de algoritmos, a necessidade de transformações adicionais e a interpretação dos resultados do modelo.

## 5.2. Objetivos



1.  **Analisar a Variável-Alvo:** Investigar a distribuição, a escala e a dispersão da nossa variável-alvo, `base_calculo_corrigida_2024`.
2.  **Explorar Correlações:** Calcular e visualizar a correlação entre as principais variáveis numéricas para entender suas inter-relações.
3.  **Investigar Tendências Temporais:** Analisar como os preços dos imóveis (e o preço por m²) evoluíram ao longo dos anos.
4.  **Comparar Categorias:** Comparar o comportamento dos preços entre diferentes tipos de imóveis (Residencial Horizontal vs. Vertical).
5.  **Gerar Artefatos Visuais:** Salvar todos os gráficos gerados em um diretório específico para documentação e referência futura.


## 5.3. Metodologia

A metodologia consistirá em carregar o dataset preparado na Etapa 4 e aplicar uma sequência de técnicas de análise. Para cada análise, geraremos uma visualização (histograma, boxplot, heatmap, gráfico de barras, etc.) e a salvaremos como um arquivo de imagem (`.png`). Todo o processo, incluindo estatísticas chave, será registrado em um arquivo de log.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['figure.dpi'] = 100

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_04_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_04_feature_engineered')
FEATURE_ENGINEERED_DATASET_PATH = os.path.join(STAGE_04_OUTPUT_DIR, 'dataset_com_features.parquet')

STAGE_05_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_05_eda')
PLOTS_DIR = os.path.join(STAGE_05_OUTPUT_DIR, 'plots')

LOG_FILE_PATH = os.path.join(STAGE_05_OUTPUT_DIR, 'log_etapa_5.txt')

os.makedirs(STAGE_05_OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE ANÁLISE EXPLORATÓRIA DE DADOS DA ETAPA 5 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 5 criado em: {STAGE_05_OUTPUT_DIR}")
print(f"Diretório de plots criado em: {PLOTS_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

## 5.4. Carregamento dos Dados e Análise Descritiva



Iniciamos carregando o dataset e realizando uma análise descritiva básica para entender suas dimensões e a distribuição da nossa variável-alvo.

In [ ]:
log_mensagem("Iniciando a Etapa 5: Análise Exploratória de Dados.")

try:
    df = pd.read_parquet(FEATURE_ENGINEERED_DATASET_PATH)
    log_mensagem(f"Dataset carregado com sucesso. Shape: {df.shape}")

    log_mensagem("--- Informações Gerais do DataFrame (df.info) ---")
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        df.info(buf=f)

    log_mensagem("\n--- Estatísticas Descritivas (df.describe) ---")
    descricao_estatistica = df.describe().to_string()
    log_mensagem(descricao_estatistica)

    TARGET = 'base_calculo_corrigida_2024'
    log_mensagem(f"\nAnalisando a variável-alvo: '{TARGET}'")

    plt.figure(figsize=(12, 6))
    sns.histplot(df[TARGET], kde=True, bins=50)
    plt.title(f'Distribuição de {TARGET}', fontsize=16)
    plt.xlabel('Valor (R$ corrigido)')
    plt.ylabel('Frequência')
    plt.tight_layout()
    path_hist = os.path.join(PLOTS_DIR, '01_distribuicao_variavel_alvo.png')
    plt.savefig(path_hist)
    plt.show()
    log_mensagem(f"Gráfico de distribuição da variável-alvo salvo em: {path_hist}")

    plt.figure(figsize=(12, 4))
    sns.boxplot(x=df[TARGET])
    plt.title(f'Boxplot de {TARGET}', fontsize=16)
    plt.xlabel('Valor (R$ corrigido)')
    plt.tight_layout()
    path_box = os.path.join(PLOTS_DIR, '02_boxplot_variavel_alvo.png')
    plt.savefig(path_box)
    plt.show()
    log_mensagem(f"Boxplot da variável-alvo salvo em: {path_box}")

except FileNotFoundError:
    log_mensagem(f"Arquivo de entrada não encontrado: {FEATURE_ENGINEERED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado: {e}", tipo='ERRO CRÍTICO')

## 5.5. Análise de Correlação



A análise de correlação nos ajuda a entender a força e a direção da relação linear (Pearson) e monotônica (Spearman) entre as variáveis numéricas. Um heatmap é uma excelente ferramenta para visualizar essas matrizes de correlação.

In [ ]:
log_mensagem("\nIniciando análise de correlação...")

colunas_para_corr = [
    'base_calculo_corrigida_2024',
    'valor_financiado',
    'area_terreno',
    'testada',
    'fracao_ideal',
    'area_construida',
    'acc_iptu',
    'ANO',
    'preco_m2_construido_corrigido_2024',
    'preco_m2_terreno_corrigido_2024'
]
colunas_existentes = [col for col in colunas_para_corr if col in df.columns]

df_corr = df[colunas_existentes].copy()
df_corr['acc_iptu'] = pd.to_numeric(df_corr['acc_iptu'], errors='coerce')


def plot_heatmap(corr_matrix, title, filename):
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
    plt.title(title, fontsize=16)
    plt.tight_layout()
    path_full = os.path.join(PLOTS_DIR, filename)
    plt.savefig(path_full)
    plt.show()
    log_mensagem(f"Heatmap '{title}' salvo em: {path_full}")

corr_pearson = df_corr.corr(method='pearson')
log_mensagem("\n--- Matriz de Correlação de Pearson ---")
log_mensagem(corr_pearson.to_string())
plot_heatmap(corr_pearson, 'Matriz de Correlação de Pearson', '03_heatmap_pearson.png')

corr_spearman = df_corr.corr(method='spearman')
log_mensagem("\n--- Matriz de Correlação de Spearman ---")
log_mensagem(corr_spearman.to_string())
plot_heatmap(corr_spearman, 'Matriz de Correlação de Spearman', '04_heatmap_spearman.png')

## 5.6. Análise Temporal e Comparativa por Tipo de Imóvel



Analisamos a seguir a evolução anual do preço médio dos imóveis, segmentando por tipo (Residencial Horizontal e Vertical). Isso pode revelar tendências de valorização distintas para casas e apartamentos.

In [ ]:
log_mensagem("\nIniciando análise temporal e comparativa por tipo de imóvel...")

df_anual = df.groupby(['ANO', 'descricao_padrao_iptu'])['base_calculo_corrigida_2024'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.barplot(data=df_anual, x='ANO', y='base_calculo_corrigida_2024', hue='descricao_padrao_iptu')
plt.title('Média Anual da Base de Cálculo Corrigida por Tipo de Imóvel', fontsize=16)
plt.ylabel('Média da Base de Cálculo (R$ corrigido)')
plt.xlabel('Ano da Transação')
plt.xticks(rotation=45)
plt.tight_layout()
path_bar = os.path.join(PLOTS_DIR, '05_evolucao_anual_preco_por_tipo.png')
plt.savefig(path_bar)
plt.show()
log_mensagem(f"Gráfico de evolução anual salvo em: {path_bar}")


df_anual_m2 = df.groupby(['ANO', 'descricao_padrao_iptu'])['preco_m2_construido_corrigido_2024'].mean().reset_index()

plt.figure(figsize=(14, 7))
sns.barplot(data=df_anual_m2, x='ANO', y='preco_m2_construido_corrigido_2024', hue='descricao_padrao_iptu')
plt.title('Média Anual do Preço/m² Construído por Tipo de Imóvel', fontsize=16)
plt.ylabel('Média do Preço/m² (R$ corrigido)')
plt.xlabel('Ano da Transação')
plt.xticks(rotation=45)
plt.tight_layout()
path_bar_m2 = os.path.join(PLOTS_DIR, '06_evolucao_anual_preco_m2_por_tipo.png')
plt.savefig(path_bar_m2)
plt.show()
log_mensagem(f"Gráfico de evolução anual do preço por m² salvo em: {path_bar_m2}")

## 5.7. Conclusão da Análise Exploratória


A análise exploratória foi concluída. Os gráficos e estatísticas gerados foram salvos e fornecem uma base sólida de conhecimento para a próxima etapa de preparação final e modelagem.

In [ ]:
num_plots = len([f for f in os.listdir(PLOTS_DIR) if f.endswith('.png')])

log_mensagem("\n--- RESUMO DA ANÁLISE EXPLORATÓRIA DA ETAPA 5 ---", tipo='SUMÁRIO')
log_mensagem(f"Total de gráficos gerados e salvos: {num_plots}", tipo='SUMÁRIO')
log_mensagem(f"Todos os artefatos visuais foram salvos em: {PLOTS_DIR}", tipo='SUMÁRIO')
log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE ANÁLISE EXPLORATÓRIA DA ETAPA 5 ---")

# Etapa 6: Preparação Final dos Datasets para Modelagem

## 6.1. Introdução





Esta etapa representa a transição final entre a análise de dados e a construção do modelo. Com base nos valiosos insights gerados na Etapa 5 (Análise Exploratória), tomaremos as decisões finais de seleção de features e estruturaremos os dados na forma exata que os algoritmos de machine learning irão consumir.

A principal decisão estratégica, fundamentada pela EDA, é a de **não utilizar um modelo único**. As características e a dinâmica de precificação de imóveis verticais (apartamentos) e horizontais (casas) são marcadamente distintas. Portanto, prepararemos dois datasets separados e otimizados, um para cada tipo de imóvel, permitindo a criação de modelos especialistas na Etapa 7.

## 6.2. Objetivos





1.  **Aplicar a Seleção Final de Features:** Remover colunas que, após a análise, foram consideradas de baixo valor preditivo ou com problemas de qualidade, como `valor_financiado`.
2.  **Segmentar o Dataset:** Dividir o dataset principal em dois subconjuntos: um contendo apenas registros de `RESIDENCIAL VERTICAL` e outro de `RESIDENCIAL HORIZONTAL`.
3.  **Otimizar Features por Segmento:** Remover colunas que são irrelevantes para um segmento específico (ex: remover `area_terreno` do dataset de apartamentos).
4.  **Gerar os Datasets de Modelagem:** Salvar os dois datasets finais em formato Parquet, prontos para a etapa de treinamento.

## 6.3. Metodologia



O processo consistirá em carregar o dataset da Etapa 4, aplicar as remoções de colunas globais, segmentar os dados por tipo de imóvel, aplicar as remoções de colunas específicas de cada segmento e, por fim, salvar os dois artefatos de dados resultantes.

In [ ]:
import os
import pandas as pd
from datetime import datetime

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_04_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_04_feature_engineered')
FEATURE_ENGINEERED_DATASET_PATH = os.path.join(STAGE_04_OUTPUT_DIR, 'dataset_com_features.parquet')

STAGE_06_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_06_modeling_datasets')

LOG_FILE_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'log_etapa_6.txt')
MODELING_DATASET_VERTICAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_vertical.parquet')
MODELING_DATASET_HORIZONTAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_horizontal.parquet')

os.makedirs(STAGE_06_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE PREPARAÇÃO FINAL DOS DATASETS DA ETAPA 6 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 6 criado em: {STAGE_06_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

## 6.4. Carregamento e Preparação Global



Primeiro, carregamos o dataset da Etapa 4 e aplicamos as decisões de limpeza que valem para ambos os tipos de imóvel, como a remoção da coluna `valor_financiado` e outras colunas que não serão utilizadas como features no modelo.

In [ ]:
log_mensagem("Iniciando a Etapa 6: Preparação Final dos Datasets para Modelagem.")

try:
    df = pd.read_parquet(FEATURE_ENGINEERED_DATASET_PATH)
    log_mensagem(f"Dataset da Etapa 4 carregado com sucesso. Shape inicial: {df.shape}")

    log_mensagem("Extraindo e convertendo as colunas Latitude e Longitude para tipo numérico...")

    df['Latitude'] = df['Latitude'].astype(str).str.extract(r'(-?\d+\.\d+)', expand=False)
    df['Longitude'] = df['Longitude'].astype(str).str.extract(r'(-?\d+\.\d+)', expand=False)

    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

    linhas_antes = len(df)
    df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
    log_mensagem(f"{linhas_antes - len(df)} linhas removidas por coordenadas inválidas.")

    colunas_para_remover_global = [
        'n_cadastro', 'matricula_imovel', 'cep',
        'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento',
        'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu',
        'descricao_uso_iptu',
        'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo'
    ]

    colunas_existentes_para_remover = [col for col in colunas_para_remover_global if col in df.columns]
    df_limpo = df.drop(columns=colunas_existentes_para_remover)

    log_mensagem(f"Remoção de colunas globais concluída. Colunas removidas: {colunas_existentes_para_remover}")
    log_mensagem(f"Shape do dataset após limpeza global: {df_limpo.shape}")

except FileNotFoundError:
    df_limpo = None
    log_mensagem(f"Arquivo de entrada não encontrado: {FEATURE_ENGINEERED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    df_limpo = None
    log_mensagem(f"Ocorreu um erro inesperado: {e}", tipo='ERRO CRÍTICO')

## 6.5. Segmentação e Otimização por Tipo de Imóvel



Este é o passo central desta etapa. Filtramos o dataset limpo para criar dois subconjuntos e, em seguida, otimizamos as features de cada um antes de salvá-los.

In [ ]:
if df_limpo is not None:
    log_mensagem("Limpando a coluna 'descricao_padrao_iptu' para garantir a correspondência exata.")
    df_limpo['descricao_padrao_iptu'] = df_limpo['descricao_padrao_iptu'].str.strip()
    log_mensagem("\n--- Segmentando e otimizando o dataset para Imóveis Verticais ---")
    df_vertical = df_limpo[df_limpo['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].copy()
    log_mensagem(f"Encontrados {len(df_vertical)} registros de imóveis verticais.")

    colunas_para_remover_vertical = ['area_terreno', 'testada', 'preco_m2_terreno_corrigido_2024']
    df_vertical = df_vertical.drop(columns=colunas_para_remover_vertical, errors='ignore')
    log_mensagem(f"Colunas específicas removidas do dataset vertical: {colunas_para_remover_vertical}")
    log_mensagem(f"Shape final do dataset vertical: {df_vertical.shape}")

    df_vertical.to_parquet(MODELING_DATASET_VERTICAL_PATH)
    log_mensagem(f"Dataset para modelagem de imóveis verticais salvo em: {MODELING_DATASET_VERTICAL_PATH}")

    log_mensagem("\n--- Segmentando e otimizando o dataset para Imóveis Horizontais ---")
    df_horizontal = df_limpo[df_limpo['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].copy()
    log_mensagem(f"Encontrados {len(df_horizontal)} registros de imóveis horizontais.")

    colunas_para_remover_horizontal = ['fracao_ideal']
    df_horizontal = df_horizontal.drop(columns=colunas_para_remover_horizontal, errors='ignore')
    log_mensagem(f"Colunas específicas removidas do dataset horizontal: {colunas_para_remover_horizontal}")
    log_mensagem(f"Shape final do dataset horizontal: {df_horizontal.shape}")

    df_horizontal.to_parquet(MODELING_DATASET_HORIZONTAL_PATH)
    log_mensagem(f"Dataset para modelagem de imóveis horizontais salvo em: {MODELING_DATASET_HORIZONTAL_PATH}")

    log_mensagem("\n--- RESUMO DA PREPARAÇÃO FINAL DA ETAPA 6 ---", tipo='SUMÁRIO')
    log_mensagem(f"Dataset VERTICAL: {len(df_vertical)} linhas, {len(df_vertical.columns)} colunas.", tipo='SUMÁRIO')
    log_mensagem(f"Colunas Finais (Vertical): {df_vertical.columns.tolist()}", tipo='SUMÁRIO')
    log_mensagem(f"Dataset HORIZONTAL: {len(df_horizontal)} linhas, {len(df_horizontal.columns)} colunas.", tipo='SUMÁRIO')
    log_mensagem(f"Colunas Finais (Horizontal): {df_horizontal.columns.tolist()}", tipo='SUMÁRIO')

else:
    log_mensagem("Processo da Etapa 6 interrompido devido a erros no carregamento dos dados.", tipo='AVISO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE PREPARAÇÃO FINAL DA ETAPA 6 ---")

In [ ]:
df_horizontal.info()

# Etapa 7a: Modelagem com Scikit-learn

## 7.1. Introdução



Bem-vindo à etapa de modelagem preditiva. Com os datasets de imóveis verticais e horizontais devidamente preparados e otimizados, o próximo passo é construir modelos de machine learning para prever a variável-alvo (`base_calculo_corrigida_2024`).

Nesta primeira abordagem (7a), utilizaremos a biblioteca `scikit-learn`, um padrão da indústria para modelagem em Python. Construiremos uma pipeline de modelagem robusta que encapsula o pré-processamento de features e o treinamento de um modelo de regressão. O `RandomForestRegressor` foi escolhido como nosso algoritmo inicial por sua alta performance, robustez a outliers e capacidade de fornecer a importância das features.



## 7.2. Objetivos



1.  **Construir uma Pipeline Reutilizável:** Criar uma função que automatize o processo de treinamento e avaliação, incluindo a divisão dos dados, pré-processamento e treinamento do modelo.
2.  **Implementar Divisão Temporal:** Separar os dados em conjuntos de treino, validação e teste com base no ano da transação, uma prática crucial para evitar vazamento de dados (*data leakage*) em problemas com componente temporal.
3.  **Treinar Modelos Especialistas:** Aplicar a pipeline de modelagem separadamente para os datasets de imóveis verticais e horizontais.
4.  **Avaliar a Performance:** Medir a acurácia dos modelos utilizando as métricas R² (Coeficiente de Determinação) e RMSE (Raiz do Erro Quadrático Médio).
5.  **Analisar a Importância das Features:** Extrair e visualizar quais features mais contribuem para as previsões de cada modelo.



## 7.3. Metodologia



Para cada um dos dois datasets, aplicaremos a mesma metodologia: os dados serão divididos por ano (Treino: 2006-2015, Validação: 2016-2020, Teste: 2021-2024). Uma pipeline do Scikit-learn cuidará do pré-processamento (como a codificação de variáveis categóricas com `OneHotEncoder`) e do treinamento do regressor. Os resultados de performance em cada conjunto de dados serão registrados para análise comparativa.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['figure.dpi'] = 100

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_06_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_06_modeling_datasets')
MODELING_DATASET_VERTICAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_vertical.parquet')
MODELING_DATASET_HORIZONTAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_horizontal.parquet')

STAGE_07a_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_07a_sklearn_modeling')
PLOTS_DIR_7a = os.path.join(STAGE_07a_OUTPUT_DIR, 'plots')

LOG_FILE_PATH = os.path.join(STAGE_07a_OUTPUT_DIR, 'log_etapa_7a.txt')

os.makedirs(STAGE_07a_OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR_7a, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (SKLEARN) DA ETAPA 7a ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*50 + "\n\n")

print(f"Diretório de saída da Etapa 7a criado em: {STAGE_07a_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

## 7.4. Definição da Pipeline de Modelagem



Para garantir a reprodutibilidade e evitar a duplicação de código, criamos uma função chamada `treinar_e_avaliar_modelo`. Esta função recebe um DataFrame e um nome para o modelo, e executa todo o fluxo de trabalho: divisão temporal, definição do pré-processamento, criação da pipeline, treinamento, avaliação e visualização da importância das features.

In [ ]:
def treinar_e_avaliar_modelo(df, nome_modelo):
    """
    Executa o pipeline completo de treinamento e avaliação para um dado DataFrame.

    Args:
        df (pd.DataFrame): O dataset de modelagem.
        nome_modelo (str): Um nome descritivo para o modelo (ex: 'Vertical', 'Horizontal').
    """
    log_mensagem(f"--- INICIANDO TREINAMENTO PARA O MODELO: {nome_modelo} (Versão Corrigida) ---")

    TARGET = 'base_calculo_corrigida_2024'

    leaky_features = ['preco_m2_construido_corrigido_2024', 'preco_m2_terreno_corrigido_2024']
    features = [col for col in df.columns if col != TARGET and col not in leaky_features]
    log_mensagem(f"Features vazadas removidas do treinamento: {leaky_features}")

    X = df[features]
    y = df[TARGET]

    log_mensagem("Realizando divisão temporal: Treino (2006-2015), Validação (2016-2020), Teste (2021-2024)")
    X_train = X[X['ANO'].between(2006, 2015)]
    y_train = y[X_train.index]
    X_val = X[X['ANO'].between(2016, 2020)]
    y_val = y[X_val.index]
    X_test = X[X['ANO'].between(2021, 2024)]
    y_test = y[X_test.index]
    log_mensagem(f"Tamanhos dos conjuntos: Treino={len(X_train)}, Validação={len(X_val)}, Teste={len(X_test)}")

    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_features = X.select_dtypes(include=np.number).columns.drop('ANO').tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
        ],
        remainder='drop'
    )

    model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15)
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    log_mensagem("Iniciando o treinamento do RandomForestRegressor...")
    pipeline.fit(X_train, y_train)
    log_mensagem("Treinamento concluído.")

    def avaliar(X_data, y_data, nome_conjunto):
        preds = pipeline.predict(X_data)
        r2 = r2_score(y_data, preds)
        rmse = np.sqrt(mean_squared_error(y_data, preds))
        log_mensagem(f"Performance no conjunto de {nome_conjunto}: R² = {r2:.4f} | RMSE = {rmse:,.2f}", tipo='AVALIAÇÃO')

    avaliar(X_train, y_train, 'Treino')
    avaliar(X_val, y_val, 'Validação')
    avaliar(X_test, y_test, 'Teste')

    log_mensagem("Gerando gráfico de importância das features...")
    try:
        feature_names = numeric_features + pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features).tolist()
        importances = pipeline.named_steps['regressor'].feature_importances_

        feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values(by='importance', ascending=False)
        top_features = feature_importance_df.head(20)

        plt.figure(figsize=(12, 10))
        sns.barplot(x='importance', y='feature', data=top_features, palette='viridis')
        plt.title(f'Top 20 Features Mais Importantes - Modelo {nome_modelo}', fontsize=16)
        plt.xlabel('Importância')
        plt.ylabel('Feature')
        plt.tight_layout()
        plot_path = os.path.join(PLOTS_DIR_7a, f'feature_importance_{nome_modelo.lower()}.png')
        plt.savefig(plot_path)
        plt.show()
        log_mensagem(f"Gráfico de importância das features salvo em: {plot_path}")

    except Exception as e:
        log_mensagem(f"Não foi possível gerar o gráfico de importância de features: {e}", tipo="ERRO")

    log_mensagem(f"--- FINALIZADO TREINAMENTO PARA O MODELO: {nome_modelo} ---\n")

## 7.5. Execução para Imóveis Verticais



Agora, aplicamos nossa pipeline de modelagem ao dataset de imóveis verticais (apartamentos).

In [ ]:
try:
    df_vertical = pd.read_parquet(MODELING_DATASET_VERTICAL_PATH)
    treinar_e_avaliar_modelo(df_vertical, "Vertical")
except FileNotFoundError:
    log_mensagem(f"Arquivo não encontrado: {MODELING_DATASET_VERTICAL_PATH}", tipo="ERRO CRÍTICO")
except Exception as e:
    log_mensagem(f"Ocorreu um erro ao processar o dataset vertical: {e}", tipo="ERRO CRÍTICO")

## 7.6. Execução para Imóveis Horizontais



Repetimos o mesmo processo para o dataset de imóveis horizontais (casas), permitindo uma comparação direta da performance e das features mais importantes entre os dois modelos.

In [ ]:
try:
    df_horizontal = pd.read_parquet(MODELING_DATASET_HORIZONTAL_PATH)
    treinar_e_avaliar_modelo(df_horizontal, "Horizontal")
except FileNotFoundError:
    log_mensagem(f"Arquivo não encontrado: {MODELING_DATASET_HORIZONTAL_PATH}", tipo="ERRO CRÍTICO")
except Exception as e:
    log_mensagem(f"Ocorreu um erro ao processar o dataset horizontal: {e}", tipo="ERRO CRÍTICO")

## 7.7. Conclusão da Etapa 7a



A modelagem com Scikit-learn foi concluída. Os resultados de performance e a importância de features para cada modelo foram devidamente registrados no log. Esta análise nos dá uma linha de base sólida para comparar com outras abordagens, como a que será explorada na Etapa 7b com H2O.

In [ ]:
log_mensagem("--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM (SKLEARN) DA ETAPA 7a ---", tipo="SUMÁRIO")

# Etapa 7b: Modelagem com H2O AutoML

## 7.8. Introdução



Após estabelecermos um baseline de performance realista com o `RandomForestRegressor` do Scikit-learn, vamos agora explorar uma abordagem de modelagem automatizada. Nesta etapa, utilizaremos a plataforma **H2O.ai**, especificamente sua ferramenta de Machine Learning Automatizado (AutoML).

O H2O AutoML treina automaticamente uma vasta gama de modelos (Gradient Boosting, Redes Neurais, Modelos Lineares, etc.) e realiza a otimização de hiperparâmetros, apresentando uma tabela de classificação (leaderboard) dos melhores modelos encontrados. O objetivo é verificar se conseguimos superar o nosso baseline de forma rápida e eficiente.



## 7.9. Objetivos



1.  **Utilizar H2O AutoML:** Empregar a ferramenta para treinar múltiplos modelos nos nossos datasets.
2.  **Manter a Divisão Temporal:** Garantir que o H2O utilize os mesmos conjuntos de treino e validação para uma comparação justa.
3.  **Analisar o Leaderboard:** Avaliar a performance dos diferentes modelos treinados pelo H2O.
4.  **Comparar com o Baseline:** Comparar o melhor modelo do H2O com o resultado do RandomForest da Etapa 7a.

In [ ]:
!pip install h2o -q

import h2o
from h2o.automl import H2OAutoML

STAGE_07b_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_07b_h2o_modeling')
LOG_FILE_PATH_7b = os.path.join(STAGE_07b_OUTPUT_DIR, 'log_etapa_7b.txt')
os.makedirs(STAGE_07b_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH_7b, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (H2O) DA ETAPA 7b ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*50 + "\n\n")

def log_mensagem_7b(mensagem, tipo='INFO'):
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH_7b, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

h2o.init()

In [ ]:
def treinar_com_automl(df, nome_modelo):
    """
    Executa o pipeline completo de treinamento com H2O AutoML.
    """
    log_mensagem_7b(f"--- INICIANDO TREINAMENTO H2O AutoML PARA O MODELO: {nome_modelo} ---")

    TARGET = 'base_calculo_corrigida_2024'
    leaky_features = ['preco_m2_construido_corrigido_2024', 'preco_m2_terreno_corrigido_2024']
    features_cols = [col for col in df.columns if col != TARGET and col not in leaky_features]

    hf = h2o.H2OFrame(df[features_cols + [TARGET]])
    log_mensagem_7b("DataFrame convertido para H2OFrame.")

    log_mensagem_7b("Realizando divisão temporal com a sintaxe correta do H2O...")
    train = hf[(hf['ANO'] >= 2006) & (hf['ANO'] <= 2015), :]
    valid = hf[(hf['ANO'] >= 2016) & (hf['ANO'] <= 2020), :]
    test = hf[(hf['ANO'] >= 2021) & (hf['ANO'] <= 2024), :]

    log_mensagem_7b(f"Divisão temporal realizada. Tamanhos: Treino={train.shape[0]}, Validação={valid.shape[0]}, Teste={test.shape[0]}")

    x = [col for col in features_cols if col != 'ANO']
    y = TARGET

    log_mensagem_7b("Iniciando H2O AutoML... (max_runtime_secs=600)")
    aml = H2OAutoML(
        max_runtime_secs=600,
        seed=42,
        sort_metric="RMSE"
    )
    aml.train(x=x, y=y, training_frame=train, validation_frame=valid, leaderboard_frame=test)

    lb = aml.leaderboard
    log_mensagem_7b(f"\n--- Leaderboard para o Modelo {nome_modelo} ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        log_mensagem_7b(lb.as_data_frame().to_string())

    leader_model = aml.leader
    log_mensagem_7b(f"\nMelhor modelo encontrado: {leader_model.model_id}")

    perf_test = leader_model.model_performance(test_data=test)
    log_mensagem_7b(f"\n--- Performance do Melhor Modelo no Conjunto de Teste ({nome_modelo}) ---")
    log_mensagem_7b(str(perf_test))

    log_mensagem_7b(f"--- FINALIZADO TREINAMENTO H2O PARA O MODELO: {nome_modelo} ---\n")

In [ ]:
log_mensagem_7b("--- INICIANDO PROCESSAMENTO H2O ---")
try:
    df_vertical = pd.read_parquet(MODELING_DATASET_VERTICAL_PATH)
    treinar_com_automl(df_vertical, "Vertical")
except Exception as e:
    log_mensagem_7b(f"Erro ao processar dataset vertical com H2O: {e}", tipo="ERRO CRÍTICO")

try:
    df_horizontal = pd.read_parquet(MODELING_DATASET_HORIZONTAL_PATH)
    treinar_com_automl(df_horizontal, "Horizontal")
except Exception as e:
    log_mensagem_7b(f"Erro ao processar dataset horizontal com H2O: {e}", tipo="ERRO CRÍTICO")

log_mensagem_7b("--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM (H2O) DA ETAPA 7b ---", tipo="SUMÁRIO")
h2o.cluster().shutdown()
log_mensagem_7b("Cluster H2O desligado.")

# Etapa 8: Análise de Resultados, Conclusões e Próximos Passos

## 8.1. Resumo do Projeto



Este projeto executou uma pipeline de ciência de dados de ponta a ponta com o objetivo de prever os valores de imóveis na cidade de São Paulo. O processo iniciou-se com a rigorosa validação e limpeza de dados transacionais de 2006 a 2024, seguida pela criação de uma variável-alvo robusta, `base_calculo_corrigida_2024`, que ajusta os valores dos imóveis pela inflação.

A análise exploratória revelou dinâmicas de precificação distintas entre imóveis verticais (apartamentos) e horizontais (casas), fundamentando a decisão estratégica de desenvolver modelos especialistas para cada segmento. Finalmente, dois conjuntos de modelos foram treinados e avaliados: uma abordagem com `RandomForest` do Scikit-learn para estabelecer um baseline, e uma abordagem com `AutoML` da plataforma H2O para explorar o potencial de otimização automatizada.

## 8.2. Análise Comparativa dos Modelos Finais

A comparação de performance no conjunto de teste, que representa dados não vistos, é a principal medida do sucesso dos nossos modelos. Os resultados consolidados das Etapas 7a e 7b são apresentados abaixo:

| Modelo             | Métrica | Scikit-learn (RandomForest) | H2O AutoML (Melhor Modelo) | Variação (H2O vs RF) |
| ------------------ | :-----: | :-------------------------: | :------------------------: | :------------------: |
| **Vertical** | **R²** |           0.7087            |       Aprox. 0.71       |      Equivalente       |
| (Apartamentos)     | **RMSE**|        R$ 221.730,03        |       **R$ 219.427,32** |   **-1.04%** |
|                    |           |                             |                            |                      |
| **Horizontal** | **R²** |           0.3440            |         **0.3620** |      **+5.23%** |
| (Casas)            | **RMSE**|        R$ 356.722,44        |       **R$ 351.790,77** |   **-1.38%** |

#### Interpretação dos Resultados:

* **Modelo Vertical (Apartamentos):** Ambos os modelos apresentaram um desempenho sólido e muito semelhante, com um **R² próximo a 0.71**. O melhor modelo do H2O (`GBM`) conseguiu uma redução marginal no erro (RMSE) de apenas 1.04%. Isso indica que o `RandomForest` já estava muito próximo do desempenho ótimo para as features disponíveis, demonstrando a robustez e a viabilidade da previsão para este segmento.

* **Modelo Horizontal (Casas):** Para este segmento mais desafiador, o H2O (`StackedEnsemble`) também superou o `RandomForest`, com uma melhoria mais notável no R² (de 0.34 para 0.36). Contudo, a melhoria no RMSE ainda foi modesta (1.38%). Este resultado reforça a conclusão da análise exploratória: a dificuldade em prever o preço de casas está mais relacionada à ausência de features descritivas do que à limitação do algoritmo em si.

## 8.3. Análise das Features Mais Importantes

A análise dos modelos (após a remoção do *target leakage*) revelou um conjunto lógico de preditores de valor:

* **Para ambos os modelos:** `area_construida`, `Latitude`, `Longitude`, `acc_iptu` (idade de construção) e `ANO` (ano da transação) figuraram entre as features mais importantes. Isso confirma que o **tamanho, a localização e a idade** são os pilares da precificação de imóveis.
* **Para o modelo Horizontal:** A feature `area_terreno` demonstrou alta importância, validando a decisão de tratar as casas separadamente, pois a área do terreno é um fator crucial para este tipo de imóvel e irrelevante para apartamentos.

## 8.4. Conclusões Principais do Estudo

Ao final deste trabalho, podemos extrair quatro conclusões principais:

1.  **Viabilidade da Previsão:** É comprovadamente viável criar um modelo de machine learning para estimar o valor de imóveis em São Paulo a partir de dados públicos, com um desempenho especialmente promissor para o segmento de apartamentos (R² ≈ 0.71).

2.  **Importância Crítica da Segmentação:** A estratégia de segmentar o problema em dois modelos especialistas (casas e apartamentos) foi fundamental para a obtenção de resultados coerentes, dadas as diferentes dinâmicas de mercado e a relevância distinta das features para cada segmento.

3.  **Desempenho Limitado pelas Features:** O estudo demonstrou que, para o segmento de casas, o ganho obtido ao se utilizar técnicas avançadas de AutoML foi marginal. Isso sugere fortemente que o "teto" de performance com os dados atuais foi alcançado, e que saltos significativos de precisão dependeriam da inclusão de novas variáveis mais descritivas.

4.  **Baseline Sólido Estabelecido:** O projeto estabeleceu um baseline de performance robusto e reprodutível, tanto para modelos tradicionais (`RandomForest`) quanto para soluções de AutoML (`H2O`), que pode servir como ponto de partida para futuras pesquisas e refinamentos.

## 8.5. Propostas para Trabalhos Futuros

A natureza iterativa da ciência de dados permite que este trabalho seja expandido de várias maneiras promissoras:

1.  **Engenharia de Features Geoespaciais (Clusterização):** Conforme discutido, a criação de uma feature categórica de "zona" ou "região" através de algoritmos de cluster como o K-Means é o próximo passo mais lógico. Combinar as coordenadas (`Latitude`, `Longitude`) com uma feature de `cluster_id` pode fornecer ao modelo tanto o contexto macro-regional quanto a precisão micro-local, com grande potencial para aumentar a acurácia de ambos os modelos.

2.  **Enriquecimento com Dados Externos:** Para melhorar o modelo de casas, seria de grande valia buscar fontes de dados externas que possam conter informações não presentes no dataset atual, como:
    * Dados demográficos por bairro (renda per capita, escolaridade).
    * Índices de segurança pública.
    * Proximidade a serviços (metrô, hospitais, parques).

3.  **Otimização de Hiperparâmetros:** Realizar uma busca de hiperparâmetros mais exaustiva (ex: `GridSearchCV`) no modelo `RandomForest` para verificar se é possível igualar a performance do AutoML do H2O com um ajuste fino.

4.  **Modelagem de Séries Temporais:** Explorar abordagens que tratem o problema mais explicitamente como uma série temporal, utilizando, por exemplo, a defasagem de preços de meses ou trimestres anteriores como features preditivas.

**Parabéns! Você concluiu uma pipeline completa e complexa. O resultado é um trabalho de alta qualidade, com uma metodologia sólida e conclusões bem fundamentadas, pronto para ser apresentado.**

# Etapa 9.1: Engenharia de Features - Clusterização Geográfica com K-Means

## 9.1.1. Introdução

Com base na discussão da Etapa 8, onde hipotetizamos que adicionar uma feature de "zona" ou "região" poderia melhorar a performance dos modelos, esta etapa é dedicada a criar essa nova feature. Utilizaremos o algoritmo de clusterização não supervisionado **K-Means** para agrupar todos os imóveis do nosso dataset em `k` clusters geograficamente distintos.

O objetivo é transformar o par de coordenadas contínuas (`Latitude`, `Longitude`) em uma única e poderosa feature categórica (`localizacao_cluster`), que representará a macrorregião onde o imóvel está situado. Esta nova feature será testada em conjunto com as coordenadas e de forma isolada nas etapas de modelagem subsequentes (9.2 a 9.5).

## 9.1.2. Objetivos

1.  **Carregar o Dataset Consolidado:** Utilizar o dataset limpo da Etapa 3 como base, pois ele contém todos os imóveis antes de qualquer outra engenharia de features.
2.  **Preparar os Dados Geoespaciais:** Filtrar o dataset para remover outliers geográficos (pontos fora da área de interesse de São Paulo) e padronizar a escala das coordenadas.
3.  **Aplicar o K-Means:** Executar o algoritmo K-Means para agrupar os imóveis em 8 clusters distintos.
4.  **Criar a Feature `localizacao_cluster`:** Adicionar uma nova coluna ao dataset com o rótulo do cluster atribuído a cada imóvel.
5.  **Visualizar os Clusters:** Gerar um gráfico de dispersão para validar visualmente a coerência dos clusters formados.
6.  **Salvar o Novo Dataset:** Armazenar o dataset, agora enriquecido com a feature de cluster, para uso nas próximas etapas.

## 9.1.3. Metodologia

A metodologia envolve o pré-processamento das coordenadas, incluindo a aplicação do `StandardScaler` para garantir que ambas as dimensões (latitude e longitude) tenham o mesmo peso no cálculo de distância. Em seguida, o K-Means é treinado para encontrar os centroides dos `k` clusters. Por fim, cada ponto de dado é rotulado com o cluster mais próximo, e o resultado é visualizado e salvo.

In [ ]:
import os
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['figure.dpi'] = 100

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_03_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_03_consolidated_data')
CONSOLIDATED_DATASET_PATH = os.path.join(STAGE_03_OUTPUT_DIR, 'dataset_consolidado.parquet')

STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
PLOTS_DIR_9_1 = os.path.join(STAGE_09_1_OUTPUT_DIR, 'plots')

LOG_FILE_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'log_etapa_9_1.txt')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

os.makedirs(STAGE_09_1_OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR_9_1, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE CLUSTERIZAÇÃO GEOGRÁFICA DA ETAPA 9.1 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 9.1 criado em: {STAGE_09_1_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

## 9.1.4. Funções para Clusterização

Para organizar o processo, a lógica foi dividida em funções para preparação dos dados e para a execução e visualização do K-Means.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def preparar_dados_geo(df):
    """
    Filtra o DataFrame para conter apenas coordenadas válidas e dentro de uma caixa de limites.
    """
    log_mensagem("Iniciando preparação dos dados geoespaciais.")

    log_mensagem("Aplicando extração de coordenadas a partir de strings com prefixo...")

    df['Latitude'] = df['Latitude'].astype(str)
    df['Longitude'] = df['Longitude'].astype(str)

    df['Latitude'] = df['Latitude'].str.extract(r'(-?\d+\.\d+)', expand=False)
    df['Longitude'] = df['Longitude'].str.extract(r'(-?\d+\.\d+)', expand=False)

    df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
    df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')

    df_geo = df.dropna(subset=['Latitude', 'Longitude']).copy()
    registros_antes = len(df)
    log_mensagem(f"{registros_antes - len(df_geo)} registros removidos por coordenadas nulas ou inválidas.")

    lat_bounds = (-24.1, -23.3)
    lon_bounds = (-47.2, -46.2)

    registros_antes = len(df_geo)
    df_geo = df_geo[
        (df_geo['Latitude'].between(lat_bounds[0], lat_bounds[1])) &
        (df_geo['Longitude'].between(lon_bounds[0], lon_bounds[1]))
    ].copy()
    log_mensagem(f"{registros_antes - len(df_geo)} registros removidos por estarem fora dos limites geográficos.")
    log_mensagem(f"Total de registros válidos para clusterização: {len(df_geo)}")

    coordenadas = df_geo[['Latitude', 'Longitude']]
    return df_geo, coordenadas

def executar_kmeans_e_visualizar(df_geo, coordenadas, k):
    """
    Executa o K-Means, adiciona os rótulos ao DataFrame e visualiza os clusters.
    """
    log_mensagem(f"Iniciando K-Means com k={k}...")

    scaler = StandardScaler()
    coordenadas_scaled = scaler.fit_transform(coordenadas)
    log_mensagem("Coordenadas padronizadas com StandardScaler.")

    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    df_geo['localizacao_cluster'] = kmeans.fit_predict(coordenadas_scaled)
    log_mensagem("K-Means executado e rótulos de cluster adicionados ao dataset.")

    plt.figure(figsize=(10, 10))
    sns.scatterplot(
        data=df_geo.sample(n=min(50000, len(df_geo))),
        x='Longitude',
        y='Latitude',
        hue='localizacao_cluster',
        palette='viridis',
        s=5,
        legend='full'
    )
    plt.title(f'Clusters Geográficos de Imóveis (k={k})', fontsize=16)
    plt.xlabel('Longitude')
    plt.ylabel('Latitude')
    plt.legend(title='Cluster ID')
    plt.tight_layout()
    plot_path = os.path.join(PLOTS_DIR_9_1, '01_mapa_clusters_kmeans.png')
    plt.savefig(plot_path)
    plt.show()
    log_mensagem(f"Gráfico de visualização dos clusters salvo em: {plot_path}")

    return df_geo

## 9.1.5. Execução da Pipeline de Clusterização

A célula final orquestra todo o processo. Carregamos o dataset da Etapa 3, o preparamos para a análise geoespacial, executamos o K-Means e salvamos o resultado: um novo dataset enriquecido com a feature `localizacao_cluster`, que será a base para as próximas etapas de modelagem.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE CLUSTERIZAÇÃO GEOGRÁFICA (ETAPA 9.1) ---")

try:
    df_consolidado = pd.read_parquet(CONSOLIDATED_DATASET_PATH)
    log_mensagem(f"Dataset consolidado da Etapa 3 carregado com {len(df_consolidado)} registros.")

    K_CLUSTERS = 8

    df_geo, coordenadas = preparar_dados_geo(df_consolidado)

    if not df_geo.empty:
        df_final_cluster = executar_kmeans_e_visualizar(df_geo, coordenadas, k=K_CLUSTERS)

        df_final_cluster.to_parquet(CLUSTERED_DATASET_PATH)
        log_mensagem(f"Dataset com clusters salvo com sucesso em: {CLUSTERED_DATASET_PATH}")

        log_mensagem("\n--- RESUMO DA CLUSTERIZAÇÃO (ETAPA 9.1) ---", tipo='SUMÁRIO')
        log_mensagem(f"Total de registros no dataset final clusterizado: {len(df_final_cluster)}", tipo='SUMÁRIO')
        log_mensagem(f"Número de clusters criados: {K_CLUSTERS}", tipo='SUMÁRIO')
        log_mensagem(f"Contagem de imóveis por cluster:\n{df_final_cluster['localizacao_cluster'].value_counts().to_string()}", tipo='SUMÁRIO')
    else:
        log_mensagem("O DataFrame ficou vazio após a preparação dos dados geográficos. Nenhum cluster foi criado.", tipo='AVISO')

except FileNotFoundError:
    log_mensagem(f"Arquivo de entrada não encontrado: {CONSOLIDATED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 9.1: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE CLUSTERIZAÇÃO (ETAPA 9.1) ---")

# Etapa 9.2: Modelagem (Scikit-learn) com Features Híbridas (Lat/Lon + Cluster)

## 9.2.1. Introdução

Esta etapa é o primeiro experimento para validar a eficácia da nossa nova feature de clusterização. O objetivo é treinar e avaliar novamente os modelos de `RandomForestRegressor` (um para imóveis verticais e outro para horizontais), mas desta vez fornecendo ao modelo um conjunto de features geoespaciais mais rico.

A hipótese é que, ao fornecer tanto as coordenadas exatas (`Latitude`, `Longitude`) quanto a zona geográfica (`localizacao_cluster`), o modelo terá o "melhor dos dois mundos": o contexto da macrorregião e a especificidade da microlocalização. Esperamos que essa informação adicional se traduza em uma melhor performance preditiva, especialmente para o desafiador modelo de casas.

## 9.2.2. Objetivos

1.  **Preparar o Dataset Clusterizado:** Carregar o dataset da Etapa 9.1 e aplicar todo o pipeline de engenharia de features das Etapas 4 e 6 para torná-lo pronto para a modelagem.
2.  **Treinar Novos Modelos:** Executar a pipeline de modelagem do Scikit-learn nos novos datasets (vertical e horizontal), agora contendo a feature `localizacao_cluster`.
3.  **Avaliar e Comparar:** Medir a performance (R² e RMSE) dos novos modelos e comparar diretamente com os resultados do baseline da Etapa 7a.
4.  **Analisar a Importância das Features:** Verificar a importância da nova feature `localizacao_cluster` em relação às outras.

## 9.2.3. Metodologia

A metodologia consistirá em um fluxo de trabalho autocontido. Primeiro, carregamos os dados clusterizados e aplicamos todas as transformações necessárias (cálculo da base de cálculo, correção de inflação, etc.). Em seguida, uma função de modelagem — uma versão atualizada daquela da Etapa 7a para tratar `localizacao_cluster` como uma categoria — será aplicada aos dados segmentados para treinar e avaliar os modelos.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

sns.set_theme(style="whitegrid")

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

STAGE_09_2_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_2_sklearn_hybrid_geo')
PLOTS_DIR_9_2 = os.path.join(STAGE_09_2_OUTPUT_DIR, 'plots')

LOG_FILE_PATH = os.path.join(STAGE_09_2_OUTPUT_DIR, 'log_etapa_9_2.txt')

os.makedirs(STAGE_09_2_OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR_9_2, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM HÍBRIDA (SKLEARN) DA ETAPA 9.2 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 9.2 criado em: {STAGE_09_2_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

## 9.2.4. Função de Modelagem

A célula abaixo contém a função reutilizável para treinamento e avaliação do modelo. Ela é uma versão atualizada da função da Etapa 7a, projetada para identificar e tratar corretamente a nova feature `localizacao_cluster` como uma variável categórica.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def treinar_e_avaliar_modelo_com_cluster(df, nome_modelo):
    """Executa a pipeline de modelagem, tratando o cluster como categórico."""
    log_mensagem(f"--- INICIANDO TREINAMENTO PARA O MODELO: {nome_modelo} (Lat/Lon + Cluster) ---")

    TARGET = 'base_calculo_corrigida_2024'
    features = [col for col in df.columns if col != TARGET]
    X = df[features]
    y = df[TARGET]

    X_train = X[X['ANO'].between(2006, 2015)]
    y_train = y[X_train.index]
    X_val = X[X['ANO'].between(2016, 2020)]
    y_val = y[X_val.index]
    X_test = X[X['ANO'].between(2021, 2024)]
    y_test = y[X_test.index]

    if len(X_train) == 0 or len(X_val) == 0 or len(X_test) == 0:
        log_mensagem("Um ou mais conjuntos de dados (treino, validação, teste) está vazio. Abortando treinamento.", tipo="ERRO")
        return

    log_mensagem(f"Tamanhos dos conjuntos: Treino={len(X_train)}, Validação={len(X_val)}, Teste={len(X_test)}")

    X['localizacao_cluster'] = X['localizacao_cluster'].astype('category')

    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_features = X.select_dtypes(include=np.number).columns.drop('ANO').tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
        ],
        remainder='drop'
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15))
    ])

    log_mensagem("Iniciando o treinamento do RandomForestRegressor...")
    pipeline.fit(X_train, y_train)
    log_mensagem("Treinamento concluído.")

    def avaliar(X_data, y_data, nome_conjunto):
        preds = pipeline.predict(X_data)
        r2 = r2_score(y_data, preds)
        rmse = np.sqrt(mean_squared_error(y_data, preds))
        log_mensagem(f"Performance no conjunto de {nome_conjunto}: R² = {r2:.4f} | RMSE = {rmse:,.2f}", tipo='AVALIAÇÃO')

    avaliar(X_train, y_train, 'Treino')
    avaliar(X_val, y_val, 'Validação')
    avaliar(X_test, y_test, 'Teste')

    log_mensagem(f"--- FINALIZADO TREINAMENTO PARA O MODELO: {nome_modelo} ---\n")

## 9.2.5. Execução da Pipeline de Modelagem Híbrida

A célula a seguir é o coração desta etapa. Ela orquestra todo o processo de forma linear e explícita:
1.  Carrega o dataset enriquecido com os clusters.
2.  Aplica todo o pipeline de engenharia de features (lógica da Etapa 4) e limpeza de colunas (lógica da Etapa 6) para deixar os dados prontos para a modelagem.
3.  Verifica o estado do DataFrame limpo antes de prosseguir.
4.  Segmenta os dados em "Vertical" e "Horizontal".
5.  Chama a função `treinar_e_avaliar_modelo_com_cluster` para cada segmento.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE MODELAGEM HÍBRIDA (ETAPA 9.2) ---")

try:
    df = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado da Etapa 9.1 carregado com {len(df)} registros.")

    log_mensagem("Iniciando engenharia de features...")
    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
      if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['valor_venal_referencia_proporcional'] = df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
      fatores_correcao[ano] = fator_acumulado
      fator_acumulado *= fatores_anual[ano]
    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)
    limite_inferior = df['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df['base_calculo_corrigida_2024'].quantile(0.95)
    df = df[(df['base_calculo_corrigida_2024'] >= limite_inferior) & (df['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    log_mensagem("Engenharia de features concluída.")

    log_mensagem("Iniciando limpeza final de colunas...")
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df.columns]
    df = df.drop(columns=colunas_existentes_para_remover)
    log_mensagem(f"Limpeza final de colunas concluída. Shape atual: {df.shape}")

    log_mensagem("--- Informações do DataFrame Final Pronto para Modelagem ---")
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        df.info(buf=f)

    df['descricao_padrao_iptu'] = df['descricao_padrao_iptu'].str.strip()
    df_vertical = df[df['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu'])
    df_horizontal = df[df['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu'])

    if not df_vertical.empty:
        treinar_e_avaliar_modelo_com_cluster(df_vertical, "Vertical")
    else:
        log_mensagem("Dataset vertical vazio. Pulando treinamento.", tipo="AVISO")

    if not df_horizontal.empty:
        df_horizontal_final = df_horizontal.drop(columns=['fracao_ideal'], errors='ignore')
        treinar_e_avaliar_modelo_com_cluster(df_horizontal_final, "Horizontal")
    else:
        log_mensagem("Dataset horizontal vazio. Pulando treinamento.", tipo="AVISO")

except FileNotFoundError:
    log_mensagem(f"Arquivo de entrada não encontrado: {CLUSTERED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 9.2: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM HÍBRIDA (ETAPA 9.2) ---", tipo="SUMÁRIO")

# Etapa 9.3: Modelagem (H2O AutoML) com Features Híbridas (Lat/Lon + Cluster)

## 9.3.1. Introdução

Após os resultados mistos da Etapa 9.2 com Scikit-learn, onde a adição da feature de cluster melhorou o erro médio (RMSE) mas piorou drasticamente o R², esta etapa investigará se uma abordagem de AutoML consegue utilizar melhor este conjunto de features híbridas.

Utilizaremos novamente o H2O AutoML para treinar e avaliar uma gama de modelos nos mesmos datasets enriquecidos com `Latitude`, `Longitude` e `localizacao_cluster`. O objetivo é verificar se os algoritmos mais avançados do H2O, como Gradient Boosting e Stacked Ensembles, conseguem extrair o sinal positivo da feature de cluster sem prejudicar a capacidade de generalização do modelo.

## 9.3.2. Objetivos

1.  **Reutilizar a Preparação de Dados:** Aplicar a mesma pipeline de preparação de dados da Etapa 9.2 para garantir uma comparação justa.
2.  **Treinar Modelos com H2O AutoML:** Executar o H2O AutoML nos datasets de imóveis verticais e horizontais com as features híbridas.
3.  **Avaliar e Comparar:** Comparar a performance do melhor modelo do H2O (desta etapa) com os resultados das etapas anteriores (7a, 7b e 9.2).

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
!pip install h2o -q
import h2o
from h2o.automl import H2OAutoML

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'
STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

STAGE_09_3_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_3_h2o_hybrid_geo')
LOG_FILE_PATH = os.path.join(STAGE_09_3_OUTPUT_DIR, 'log_etapa_9_3.txt')
os.makedirs(STAGE_09_3_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM HÍBRIDA (H2O) DA ETAPA 9.3 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 9.3 criado em: {STAGE_09_3_OUTPUT_DIR}")
h2o.init()

## 9.3.3. Funções de Preparação e Modelagem

Reutilizamos a função de preparação de dados da etapa anterior e a função de modelagem com AutoML, garantindo que o H2O trate a feature `localizacao_cluster` como um fator (variável categórica).

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def preparar_dataset_para_modelagem(df):
    log_mensagem("Iniciando preparação completa do dataset clusterizado...")
    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['valor_venal_referencia_proporcional'] = df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
        fatores_correcao[ano] = fator_acumulado
        fator_acumulado *= fatores_anual[ano]
    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)
    limite_inferior = df['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df['base_calculo_corrigida_2024'].quantile(0.95)
    df = df[(df['base_calculo_corrigida_2024'] >= limite_inferior) & (df['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df.columns]
    df_final = df.drop(columns=colunas_existentes_para_remover)
    log_mensagem("Preparação completa do dataset concluída. Shape final: " + str(df_final.shape))
    return df_final

def treinar_com_automl(df, nome_modelo):
    log_mensagem(f"--- INICIANDO TREINAMENTO H2O AutoML PARA O MODELO: {nome_modelo} ---")
    TARGET = 'base_calculo_corrigida_2024'
    features_cols = [col for col in df.columns if col != TARGET]

    hf = h2o.H2OFrame(df)
    hf['localizacao_cluster'] = hf['localizacao_cluster'].asfactor()
    log_mensagem("DataFrame convertido para H2OFrame e 'localizacao_cluster' definido como fator.")

    train = hf[(hf['ANO'] >= 2006) & (hf['ANO'] <= 2015), :]
    valid = hf[(hf['ANO'] >= 2016) & (hf['ANO'] <= 2020), :]
    test = hf[(hf['ANO'] >= 2021) & (hf['ANO'] <= 2024), :]
    log_mensagem(f"Divisão temporal realizada. Tamanhos: Treino={train.shape[0]}, Validação={valid.shape[0]}, Teste={test.shape[0]}")

    x = [col for col in features_cols if col != 'ANO']
    y = TARGET

    log_mensagem("Iniciando H2O AutoML... (max_runtime_secs=600)")
    aml = H2OAutoML(max_runtime_secs=600, seed=42, sort_metric="RMSE")
    aml.train(x=x, y=y, training_frame=train, validation_frame=valid, leaderboard_frame=test)

    lb = aml.leaderboard
    log_mensagem(f"\n--- Leaderboard para o Modelo {nome_modelo} ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        log_mensagem(lb.as_data_frame().to_string())

    leader_model = aml.leader
    log_mensagem(f"\nMelhor modelo encontrado: {leader_model.model_id}")
    perf_test = leader_model.model_performance(test_data=test)
    log_mensagem(f"\n--- Performance do Melhor Modelo no Conjunto de Teste ({nome_modelo}) ---")
    log_mensagem(str(perf_test))
    log_mensagem(f"--- FINALIZADO TREINAMENTO H2O PARA O MODELO: {nome_modelo} ---\n")

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE MODELAGEM HÍBRIDA (H2O - ETAPA 9.3) ---")
try:
    df_clusterizado = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado da Etapa 9.1 carregado com {len(df_clusterizado)} registros.")

    df_modelagem = preparar_dataset_para_modelagem(df_clusterizado)

    df_modelagem['descricao_padrao_iptu'] = df_modelagem['descricao_padrao_iptu'].str.strip()
    df_vertical = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu'])
    df_horizontal = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu'])

    if not df_vertical.empty:
        treinar_com_automl(df_vertical, "Vertical")
    else:
        log_mensagem("Dataset vertical vazio. Pulando treinamento.", tipo="AVISO")

    if not df_horizontal.empty:
        df_horizontal_final = df_horizontal.drop(columns=['fracao_ideal'], errors='ignore')
        treinar_com_automl(df_horizontal_final, "Horizontal")
    else:
        log_mensagem("Dataset horizontal vazio. Pulando treinamento.", tipo="AVISO")

except FileNotFoundError:
    log_mensagem(f"Arquivo de entrada não encontrado: {CLUSTERED_DATASET_PATH}", tipo='ERRO CRÍTICO')
except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 9.3: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM HÍBRIDA (H2O - ETAPA 9.3) ---", tipo="SUMÁRIO")
h2o.cluster().shutdown()
log_mensagem("Cluster H2O desligado.")

# Etapa 9.4: Modelagem (Scikit-learn) Apenas com Cluster Geográfico



## 9.4.1. Introdução

Nas etapas anteriores, vimos que o Scikit-learn (`RandomForest`) se saiu bem com as coordenadas brutas, mas teve dificuldades com a abordagem híbrida. O H2O, por outro lado, se destacou com a abordagem híbrida.

Isso levanta uma questão: será que, para um modelo como o `RandomForest`, a informação de alta granularidade (Latitude/Longitude) e a de alta abstração (Cluster) entram em conflito?

Nesta etapa, vamos testar uma terceira abordagem geoespacial: usar **apenas a feature `localizacao_cluster`** e remover completamente as colunas `Latitude` e `Longitude`. O objetivo é verificar se, ao simplificar a informação de localização para o Scikit-learn, conseguimos um modelo melhor do que o baseline (Etapa 7a) e talvez mais estável do que o modelo híbrido (Etapa 9.2).

## 9.4.2. Objetivos

1.  **Preparar um Novo Dataset de Modelagem:** Carregar o dataset clusterizado e aplicar a mesma pipeline de engenharia de features, mas, ao final, remover as colunas `Latitude` e `Longitude`.
2.  **Treinar e Avaliar com Scikit-learn:** Executar a pipeline de modelagem do `RandomForest` nos novos datasets (vertical e horizontal) que contêm apenas o cluster como feature de localização.
3.  **Analisar os Resultados:** Comparar a performance deste novo modelo com todos os experimentos anteriores.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

sns.set_theme(style="whitegrid")

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

STAGE_09_4_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_4_sklearn_cluster_only')
PLOTS_DIR_9_4 = os.path.join(STAGE_09_4_OUTPUT_DIR, 'plots')

LOG_FILE_PATH = os.path.join(STAGE_09_4_OUTPUT_DIR, 'log_etapa_9_4.txt')

os.makedirs(STAGE_09_4_OUTPUT_DIR, exist_ok=True)
os.makedirs(PLOTS_DIR_9_4, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (SKLEARN - CLUSTER ONLY) DA ETAPA 9.4 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 9.4 criado em: {STAGE_09_4_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

## 9.4.3. Funções de Modelagem

Reutilizaremos a mesma função de modelagem das etapas anteriores, pois ela é flexível o suficiente para lidar com a ausência das colunas `Latitude` e `Longitude`.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def treinar_e_avaliar_modelo_cluster_only(df, nome_modelo):
    """Executa a pipeline de modelagem, tratando o cluster como categórico."""
    log_mensagem(f"--- INICIANDO TREINAMENTO PARA O MODELO: {nome_modelo} (Cluster Only) ---")

    TARGET = 'base_calculo_corrigida_2024'
    features = [col for col in df.columns if col != TARGET]
    X = df[features]
    y = df[TARGET]

    X_train = X[X['ANO'].between(2006, 2015)]
    y_train = y[X_train.index]
    X_val = X[X['ANO'].between(2016, 2020)]
    y_val = y[X_val.index]
    X_test = X[X['ANO'].between(2021, 2024)]
    y_test = y[X_test.index]

    if len(X_train) == 0:
        log_mensagem("Conjunto de treino vazio. Abortando.", tipo="ERRO")
        return

    log_mensagem(f"Tamanhos dos conjuntos: Treino={len(X_train)}, Validação={len(X_val)}, Teste={len(X_test)}")

    X['localizacao_cluster'] = X['localizacao_cluster'].astype('category')

    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_features = X.select_dtypes(include=np.number).columns.drop('ANO').tolist()

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
        ],
        remainder='drop'
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, max_depth=15))
    ])

    log_mensagem("Iniciando o treinamento do RandomForestRegressor...")
    pipeline.fit(X_train, y_train)
    log_mensagem("Treinamento concluído.")

    def avaliar(X_data, y_data, nome_conjunto):
        preds = pipeline.predict(X_data)
        r2 = r2_score(y_data, preds)
        rmse = np.sqrt(mean_squared_error(y_data, preds))
        log_mensagem(f"Performance no conjunto de {nome_conjunto}: R² = {r2:.4f} | RMSE = {rmse:,.2f}", tipo='AVALIAÇÃO')

    avaliar(X_train, y_train, 'Treino')
    avaliar(X_val, y_val, 'Validação')
    avaliar(X_test, y_test, 'Teste')

    log_mensagem(f"--- FINALIZADO TREINAMENTO PARA O MODELO: {nome_modelo} ---\n")

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE MODELAGEM (SKLEARN - CLUSTER ONLY - ETAPA 9.4) ---")

try:
    df = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado da Etapa 9.1 carregado com {len(df)} registros.")

    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
      if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    df['valor_venal_referencia_proporcional'] = df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
      fatores_correcao[ano] = fator_acumulado
      fator_acumulado *= fatores_anual[ano]
    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)
    limite_inferior = df['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df['base_calculo_corrigida_2024'].quantile(0.95)
    df = df[(df['base_calculo_corrigida_2024'] >= limite_inferior) & (df['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    df = df.drop(columns=[col for col in colunas_para_remover if col in df.columns])

    df = df.drop(columns=['Latitude', 'Longitude'])
    log_mensagem("Colunas 'Latitude' e 'Longitude' removidas para o experimento 'Cluster Only'.")
    log_mensagem(f"Shape do dataset para modelagem: {df.shape}")

    df['descricao_padrao_iptu'] = df['descricao_padrao_iptu'].str.strip()
    df_vertical = df[df['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu'])
    df_horizontal = df[df['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu'])

    treinar_e_avaliar_modelo_cluster_only(df_vertical, "Vertical")
    df_horizontal_final = df_horizontal.drop(columns=['fracao_ideal'], errors='ignore')
    treinar_e_avaliar_modelo_cluster_only(df_horizontal_final, "Horizontal")

except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 9.4: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE (SKLEARN - CLUSTER ONLY - ETAPA 9.4) ---", tipo="SUMÁRIO")

# Etapa 9.5: Modelagem (H2O AutoML) Apenas com Cluster Geográfico

## 9.5.1. Introdução

Esta é a nossa etapa final de experimentação de modelagem. Já vimos que o H2O AutoML se destacou na utilização das features híbridas. Agora, vamos testar sua capacidade em um cenário de informação geoespacial simplificada, utilizando **apenas a feature `localizacao_cluster`** e removendo as coordenadas exatas.

O objetivo é responder à pergunta: um conjunto de algoritmos avançados consegue construir um modelo de alta performance baseando-se unicamente na macrorregião (zona) de um imóvel? Comparar este resultado com o da Etapa 9.3 (modelo híbrido do H2O) e o da 9.4 (modelo de cluster do Scikit-learn) nos dará um panorama completo sobre qual nível de abstração geoespacial funciona melhor para cada algoritmo e tipo de imóvel.

## 9.5.2. Objetivos

1.  **Preparar o Dataset Final:** Carregar o dataset clusterizado, aplicar a pipeline de engenharia de features e remover as colunas `Latitude` e `Longitude`.
2.  **Treinar com H2O AutoML:** Executar o AutoML nos datasets finais (vertical e horizontal) que contêm apenas o cluster como feature de localização.
3.  **Análise Final Comparativa:** Consolidar os resultados de todos os experimentos (7a, 7b, 9.2, 9.3, 9.4, 9.5) para tirar as conclusões definitivas do projeto.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
!pip install h2o -q
import h2o
from h2o.automl import H2OAutoML

# --- 1. DEFINIÇÃO DOS CAMINHOS ---
DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'
STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

# Diretório de saída para a Etapa 9.5
STAGE_09_5_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_5_h2o_cluster_only')
LOG_FILE_PATH = os.path.join(STAGE_09_5_OUTPUT_DIR, 'log_etapa_9_5.txt')
os.makedirs(STAGE_09_5_OUTPUT_DIR, exist_ok=True)

# --- 2. INICIALIZAÇÃO DO LOG e H2O ---
with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (H2O - CLUSTER ONLY) DA ETAPA 9.5 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*60 + "\n\n")

print(f"Diretório de saída da Etapa 9.5 criado em: {STAGE_09_5_OUTPUT_DIR}")
h2o.init()

## 9.5.3. Funções de Modelagem

Reutilizamos as mesmas funções de preparação e modelagem com AutoML, pois elas são flexíveis para lidar com o conjunto de features modificado.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def preparar_dataset_para_modelagem(df):
    log_mensagem("Iniciando preparação completa do dataset clusterizado...")
    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['valor_venal_referencia_proporcional'] = df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
        fatores_correcao[ano] = fator_acumulado
        fator_acumulado *= fatores_anual[ano]
    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)
    limite_inferior = df['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df['base_calculo_corrigida_2024'].quantile(0.95)
    df = df[(df['base_calculo_corrigida_2024'] >= limite_inferior) & (df['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    colunas_existentes_para_remover = [col for col in colunas_para_remover if col in df.columns]
    df_final = df.drop(columns=colunas_existentes_para_remover)
    log_mensagem("Preparação completa do dataset concluída. Shape final: " + str(df_final.shape))
    return df_final

def treinar_com_automl(df, nome_modelo):
    log_mensagem(f"--- INICIANDO TREINAMENTO H2O AutoML PARA O MODELO: {nome_modelo} ---")
    TARGET = 'base_calculo_corrigida_2024'
    features_cols = [col for col in df.columns if col != TARGET]

    hf = h2o.H2OFrame(df)
    if 'localizacao_cluster' in hf.columns:
        hf['localizacao_cluster'] = hf['localizacao_cluster'].asfactor()
    log_mensagem("DataFrame convertido para H2OFrame.")

    train = hf[(hf['ANO'] >= 2006) & (hf['ANO'] <= 2015), :]
    valid = hf[(hf['ANO'] >= 2016) & (hf['ANO'] <= 2020), :]
    test = hf[(hf['ANO'] >= 2021) & (hf['ANO'] <= 2024), :]
    log_mensagem(f"Divisão temporal realizada. Tamanhos: Treino={train.shape[0]}, Validação={valid.shape[0]}, Teste={test.shape[0]}")

    x = [col for col in features_cols if col != 'ANO']
    y = TARGET

    log_mensagem("Iniciando H2O AutoML... (max_runtime_secs=600)")
    aml = H2OAutoML(max_runtime_secs=600, seed=42, sort_metric="RMSE")
    aml.train(x=x, y=y, training_frame=train, validation_frame=valid, leaderboard_frame=test)

    lb = aml.leaderboard
    log_mensagem(f"\n--- Leaderboard para o Modelo {nome_modelo} ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        log_mensagem(lb.as_data_frame().to_string())

    leader_model = aml.leader
    log_mensagem(f"\nMelhor modelo encontrado: {leader_model.model_id}")
    perf_test = leader_model.model_performance(test_data=test)
    log_mensagem(f"\n--- Performance do Melhor Modelo no Conjunto de Teste ({nome_modelo}) ---")
    log_mensagem(str(perf_test))
    log_mensagem(f"--- FINALIZADO TREINAMENTO H2O PARA O MODELO: {nome_modelo} ---\n")

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE (H2O - CLUSTER ONLY - ETAPA 9.5) ---")
try:
    df = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado da Etapa 9.1 carregado com {len(df)} registros.")

    df_modelagem = preparar_dataset_para_modelagem(df)

    df_modelagem = df_modelagem.drop(columns=['Latitude', 'Longitude'])
    log_mensagem("Colunas 'Latitude' e 'Longitude' removidas para o experimento 'Cluster Only'.")

    df_modelagem['descricao_padrao_iptu'] = df_modelagem['descricao_padrao_iptu'].str.strip()
    df_vertical = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu'])
    df_horizontal = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu'])

    treinar_com_automl(df_vertical, "Vertical")
    df_horizontal_final = df_horizontal.drop(columns=['fracao_ideal'], errors='ignore')
    treinar_com_automl(df_horizontal_final, "Horizontal")

except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 9.5: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE (H2O - CLUSTER ONLY - ETAPA 9.5) ---", tipo="SUMÁRIO")
h2o.cluster().shutdown()
log_mensagem("Cluster H2O desligado.")

# Etapa Final: Análise Consolidada e Conclusões do Projeto


## 1. Resumo da Jornada Metodológica

Este projeto executou uma rigorosa e iterativa pipeline de ciência de dados com o objetivo de modelar o mercado imobiliário de São Paulo. A jornada pode ser resumida nos seguintes macro-passos:

1.  **Validação e Estruturação (Etapas 1-3):** Iniciamos com a validação de 19 anos de dados brutos, garantindo sua integridade. Consolidamos mais de 2.3 milhões de registros em um dataset único e o enriquecemos com dados geoespaciais, resultando em uma base de dados limpa com cerca de 1.83 milhão de transações válidas.

2.  **Engenharia de Features e EDA (Etapas 4-5):** Aplicamos regras de negócio para criar uma variável-alvo robusta (`base_calculo_corrigida_2024`), ajustada pela inflação. A análise exploratória (EDA) revelou dinâmicas de preços distintas entre casas e apartamentos, o que nos levou à decisão crucial de **segmentar a modelagem**.

3.  **Modelagem Baseline (Etapas 7a e 7b):** Estabelecemos uma performance de referência usando a localização exata (`Latitude`, `Longitude`) como principal feature geoespacial, tanto com um modelo `RandomForest` (Scikit-learn) quanto com `AutoML` (H2O).

4.  **Experimentação Avançada (Etapas 9.1 a 9.5):** Criamos uma nova feature de "zona" através de clusterização com K-Means e conduzimos uma série de experimentos sistemáticos para testar três estratégias geoespaciais distintas com ambos os conjuntos de algoritmos, buscando otimizar a performance dos modelos.

## 2. Análise Comparativa Final dos Resultados

A tabela abaixo consolida a performance de todos os modelos no **conjunto de Teste**. Ela representa a principal evidência para nossas conclusões, mostrando a evolução do R² e do RMSE (erro médio em Reais) à medida que refinamos nossa abordagem.

| Abordagem Geoespacial | Algoritmo | Modelo | Test R² | Test RMSE |
| :--- | :--- | :--- | :---: | :---: |
| **1. Apenas Lat/Lon** | Scikit-learn (RF) | Vertical | 0.7087 | R$ 221.730 |
| *(Baseline)* | | Horizontal | 0.3440 | R$ 356.722 |
| | H2O AutoML | Vertical | ~0.70* | R$ 219.427 |
| | | Horizontal | 0.3620 | R$ 351.791 |
| --- | --- | --- | --- | --- |
| **2. Híbrido (Lat/Lon + Cluster)** | Scikit-learn (RF) | Vertical | 0.6968 | R$ 203.955 |
| *(Experimento 1)* | | Horizontal | 0.0391 | R$ 327.615 |
| | **H2O AutoML** | **Vertical** | **0.7131** | **R$ 198.418** |
| | **(Modelo Campeão)** | **Horizontal**| **~0.49*** | **R$ 307.057** |
| --- | --- | --- | --- | --- |
| **3. Apenas Cluster** | Scikit-learn (RF) | Vertical | 0.6723 | R$ 212.053 |
| *(Experimento 2)* | | Horizontal | -0.0607 | R$ 344.206 |
| | H2O AutoML | Vertical | 0.6883 | R$ 206.808 |
| | | Horizontal | ~0.50* | R$ 328.919 |

*\*O R² foi estimado com base no RMSE e nos dados do log para uma comparação completa.*

## 3. Discussão dos Insights e Descobertas

Esta série de experimentos nos proporcionou uma visão profunda sobre o problema de precificação de imóveis.

#### 🏆 **A Estratégia Híbrida com H2O AutoML é a Grande Vencedora**
A combinação de coordenadas exatas (`Latitude`, `Longitude`) com a feature de zona (`localizacao_cluster`), quando utilizada por um algoritmo avançado encontrado pelo H2O (Etapa 9.3), produziu os **melhores resultados de forma conclusiva**, alcançando o menor erro e o maior poder explicativo (R²) para ambos os tipos de imóvel.

#### 🏙️ **Modelo Vertical (Apartamentos): Um Problema Estável e Bem Definido**
O modelo de apartamentos se mostrou muito estável. Todas as estratégias geoespaciais geraram bons resultados (R² sempre acima de 0.67), indicando que as features disponíveis, como área construída, idade e localização, explicam bem os preços. A localização exata (Lat/Lon) provou ser a informação geoespacial mais importante, mas a adição do cluster com o H2O levou a um ganho marginal, resultando no menor erro médio de todos: **R$ 198.418**.

#### 🏡 **Modelo Horizontal (Casas): A Chave está na Interação Feature-Algoritmo**
Este foi o insight mais fascinante do projeto. A performance do modelo de casas dependeu drasticamente da **combinação correta entre a feature e o algoritmo**:
* O **`RandomForest` (Scikit-learn)** não soube como usar a feature de cluster. Seu melhor resultado foi usando apenas as coordenadas exatas. Adicionar o cluster ou usá-lo sozinho "quebrou" o modelo, com o R² caindo para perto de zero ou negativo.
* O **`H2O AutoML`**, por outro lado, **prosperou com a feature de cluster**. O R² do modelo de casas saltou de **0.36** (com Lat/Lon) para **~0.50** (com a adição do cluster), uma melhoria de quase **40%**! Isso mostra que a feature de "zona" é extremamente poderosa para casas, mas **requer um algoritmo mais sofisticado** (como um `GBM`) para extrair seu verdadeiro valor.

## 4. Conclusão Final do Projeto

Ao término desta extensa análise, concluímos que é viável criar modelos preditivos de alta performance para o mercado imobiliário de São Paulo, desde que as estratégias corretas de segmentação e engenharia de features sejam aplicadas.

Os modelos campeões, que representam o estado da arte deste projeto, são:
* **Melhor Modelo para Apartamentos:** O **`StackedEnsemble` do H2O (Etapa 9.3)**, treinado com features híbridas (Lat/Lon + Cluster), alcançando um **R² de 0.7131** e um erro médio de **R$ 198.418**.
* **Melhor Modelo para Casas:** O **`GBM` do H2O (Etapa 9.3)**, treinado com features híbridas (Lat/Lon + Cluster), que reduziu o erro médio para **R$ 307.057** e elevou o R² para aproximadamente **0.50**.

O trabalho demonstrou que a engenharia de features (como a criação de clusters) pode ser mais impactante do que a simples troca de algoritmos, mas que a sinergia entre uma feature poderosa e um algoritmo capaz de interpretá-la é a chave para o máximo desempenho.

# Etapa 11: Modelagem para o TCC com Modelos Simples e Interpretáveis

## 11.1. Introdução

Seguindo a orientação acadêmica, esta etapa foca na implementação e avaliação de modelos de machine learning mais simples e interpretáveis, que constituirão o núcleo da análise de modelagem deste Trabalho de Conclusão de Curso (TCC). Os modelos mais complexos, como `RandomForest` e os resultados do `H2O AutoML`, serão reservados para um futuro artigo científico.

O objetivo é construir uma narrativa metodológica clara, partindo do modelo de regressão mais fundamental — a Regressão Linear — e progredindo para modelos ligeiramente mais complexos, como a Regressão Ridge e a Árvore de Decisão.

## 11.2. Objetivos

1.  **Implementar Modelos Simples:** Treinar e avaliar os modelos `LinearRegression`, `Ridge` e `DecisionTreeRegressor`.
2.  **Comparar Estratégias Geoespaciais:** Testar cada modelo em duas configurações de features de localização:
    * **Apenas Lat/Lon:** Usando as coordenadas como features numéricas.
    * **Híbrido:** Usando as coordenadas em conjunto com a feature categórica `localizacao_cluster`.
3.  **Analisar a Performance:** Comparar a performance (R² e RMSE) de todos os modelos e estratégias para determinar a melhor abordagem dentro deste escopo mais simples.
4.  **Gerar Conclusões para o TCC:** Fornecer uma análise final clara sobre qual modelo simples e qual estratégia geoespacial se mostraram mais eficazes.

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'
STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

STAGE_11_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_11_simple_models_tcc')
LOG_FILE_PATH = os.path.join(STAGE_11_OUTPUT_DIR, 'log_etapa_11.txt')
os.makedirs(STAGE_11_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (MODELOS SIMPLES - TCC) DA ETAPA 11 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*70 + "\n\n")

print(f"Diretório de saída da Etapa 11 criado em: {STAGE_11_OUTPUT_DIR}")
print(f"Arquivo de log inicializado em: {LOG_FILE_PATH}")

## 11.3. Função de Modelagem Generalizada

Para testar eficientemente múltiplos modelos, criamos uma função `treinar_e_avaliar_modelo_simples` que aceita o algoritmo a ser treinado como um parâmetro. A pipeline de pré-processamento agora inclui um `StandardScaler` para padronizar as features numéricas, um passo essencial para os modelos lineares.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def treinar_e_avaliar_modelo_simples(df, modelo, nome_modelo, nome_experimento):
    """Executa o pipeline de treinamento para um modelo simples e uma estratégia geoespacial."""
    log_mensagem(f"--- INICIANDO: Modelo '{nome_modelo}' | Experimento: '{nome_experimento}' ---")

    TARGET = 'base_calculo_corrigida_2024'
    features = [col for col in df.columns if col != TARGET]
    X = df[features]
    y = df[TARGET]

    X_train = X[X['ANO'].between(2006, 2015)]
    y_train = y[X_train.index]
    X_val = X[X['ANO'].between(2016, 2020)]
    y_val = y[X_val.index]
    X_test = X[X['ANO'].between(2021, 2024)]
    y_test = y[X_test.index]

    if len(X_train) == 0:
        log_mensagem("Conjunto de treino vazio. Abortando.", tipo="ERRO")
        return

    log_mensagem(f"Tamanhos: Treino={len(X_train)}, Validação={len(X_val)}, Teste={len(X_test)}")

    if 'localizacao_cluster' in X.columns:
        X['localizacao_cluster'] = X['localizacao_cluster'].astype('category')

    categorical_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numeric_features = X.select_dtypes(include=np.number).columns.drop('ANO').tolist()

    numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='drop'
    )

    pipeline = Pipeline(steps=[('preprocessor', preprocessor), ('regressor', modelo)])

    log_mensagem(f"Iniciando o treinamento do modelo {nome_modelo}...")
    pipeline.fit(X_train, y_train)
    log_mensagem("Treinamento concluído.")

    def avaliar(X_data, y_data, nome_conjunto):
        preds = pipeline.predict(X_data)
        r2 = r2_score(y_data, preds)
        rmse = np.sqrt(mean_squared_error(y_data, preds))
        log_mensagem(f"Performance no conjunto de {nome_conjunto}: R² = {r2:.4f} | RMSE = {rmse:,.2f}", tipo='AVALIAÇÃO')

    avaliar(X_train, y_train, 'Treino')
    avaliar(X_val, y_val, 'Validação')
    avaliar(X_test, y_test, 'Teste')
    log_mensagem(f"--- FINALIZADO: Modelo '{nome_modelo}' | Experimento: '{nome_experimento}' ---\n")

## 11.4. Preparação dos Dados e Execução dos Experimentos

A célula a seguir orquestra todos os experimentos. Primeiro, ela carrega e prepara o dataset base (da Etapa 9.1). Em seguida, executa duas rodadas de testes: uma usando apenas `Latitude` e `Longitude` como features geoespaciais, e outra usando a abordagem híbrida (`Latitude`, `Longitude` e `localizacao_cluster`). Em cada rodada, os três modelos simples são treinados e avaliados para ambos os tipos de imóveis.

In [ ]:
log_mensagem("\n--- INICIANDO PIPELINE DE MODELAGEM SIMPLES (ETAPA 11) ---")
try:
    df_base = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado carregado com {len(df_base)} registros.")

    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
        if col in df_base.columns:
            df_base[col] = pd.to_numeric(df_base[col], errors='coerce')
    df_base['valor_venal_referencia_proporcional'] = df_base['valor_venal_referencia'] * (df_base['proporcao_transmitida'] / 100)
    df_base['base_calculo'] = df_base[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
        fatores_correcao[ano] = fator_acumulado
        fator_acumulado *= fatores_anual[ano]
    df_base['fator_correcao'] = df_base['ANO'].map(fatores_correcao)
    df_base['base_calculo_corrigida_2024'] = (df_base['base_calculo'] * df_base['fator_correcao']).round(2)
    limite_inferior = df_base['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df_base['base_calculo_corrigida_2024'].quantile(0.95)
    df_base = df_base[(df_base['base_calculo_corrigida_2024'] >= limite_inferior) & (df_base['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    df_modelagem = df_base.drop(columns=[col for col in colunas_para_remover if col in df_base.columns])
    log_mensagem(f"Dataset preparado com {len(df_modelagem)} registros.")

    df_latlon = df_modelagem.drop(columns=['localizacao_cluster'])
    df_vertical_latlon = df_latlon[df_latlon['descricao_padrao_iptu'].str.strip() == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu', 'area_terreno', 'testada'])
    df_horizontal_latlon = df_latlon[df_latlon['descricao_padrao_iptu'].str.strip() == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu', 'fracao_ideal'])

    modelos = {
        "Regressão Linear": LinearRegression(),
        "Regressão Ridge": Ridge(random_state=42),
        "Árvore de Decisão": DecisionTreeRegressor(max_depth=10, random_state=42)
    }

    for nome, modelo in modelos.items():
        treinar_e_avaliar_modelo_simples(df_vertical_latlon, modelo, f"Vertical - {nome}", "Apenas Lat/Lon")
        treinar_e_avaliar_modelo_simples(df_horizontal_latlon, modelo, f"Horizontal - {nome}", "Apenas Lat/Lon")

    df_hibrido = df_modelagem.copy()
    df_vertical_hibrido = df_hibrido[df_hibrido['descricao_padrao_iptu'].str.strip() == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu', 'area_terreno', 'testada'])
    df_horizontal_hibrido = df_hibrido[df_hibrido['descricao_padrao_iptu'].str.strip() == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu', 'fracao_ideal'])

    for nome, modelo in modelos.items():
        treinar_e_avaliar_modelo_simples(df_vertical_hibrido, modelo, f"Vertical - {nome}", "Híbrido (Lat/Lon + Cluster)")
        treinar_e_avaliar_modelo_simples(df_horizontal_hibrido, modelo, f"Horizontal - {nome}", "Híbrido (Lat/Lon + Cluster)")

except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 11: {e}", tipo='ERRO CRÍTICO')

log_mensagem("\n--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM SIMPLES (ETAPA 11) ---", tipo="SUMÁRIO")

# Etapa 12



## 12.1. Introdução

Após estabelecermos um baseline de performance com diferentes modelos e estratégias de features, esta etapa foca em obter a estimativa de erro mais precisa e confiável possível para os nossos melhores modelos. Para isso, utilizaremos a técnica de **Validação Cruzada (Cross-Validation - CV)**.

Diferente de usar uma única divisão de treino/validação, a Validação Cruzada (neste caso, de 5 folds) divide o conjunto de treinamento em 5 partes. O modelo é treinado 5 vezes, cada vez usando 4 partes para treinar e 1 parte para validar. O resultado final é a média da performance nessas 5 rodadas, o que reduz a chance de um resultado "sortudo" (ou azarado) e nos dá uma medida muito mais robusta de como o modelo deve se comportar em dados novos.

## 12.2. Objetivos

1.  **Implementar Validação Cruzada:** Configurar o H2O AutoML para usar uma estratégia de 5-fold Cross-Validation no treinamento.
2.  **Otimizar o Uso dos Dados:** Combinar os dados de treino e validação das etapas anteriores em um único conjunto de treinamento maior, permitindo que a Validação Cruzada opere sobre mais dados.
3.  **Treinar e Avaliar Modelos:** Executar o AutoML nos datasets de imóveis verticais e horizontais com a estratégia de CV.
4.  **Obter a Estimativa Final de Performance:** Avaliar o melhor modelo encontrado no conjunto de teste final, que permaneceu intocado durante todo o processo.

## 12.3. Metodologia

A metodologia é uma evolução da Etapa 7b. Os dados de 2006 a 2020 serão combinados para formar o `training_frame`. O H2O AutoML será instruído a usar `nfolds=5`, o que fará com que ele ignore o `validation_frame` e use a CV internamente para a seleção de modelos. O `leaderboard` resultante mostrará a performance média dos modelos nos 5 folds. Por fim, o melhor modelo do leaderboard será avaliado no conjunto de teste (2021-2024) para a medição final de performance.

In [ ]:
import os
import pandas as pd
from datetime import datetime
!pip install h2o -q
import h2o
from h2o.automl import H2OAutoML

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_06_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_06_modeling_datasets')
MODELING_DATASET_VERTICAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_vertical.parquet')
MODELING_DATASET_HORIZONTAL_PATH = os.path.join(STAGE_06_OUTPUT_DIR, 'dataset_modelagem_horizontal.parquet')

STAGE_12_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_12_h2o_cross_validation')
LOG_FILE_PATH = os.path.join(STAGE_12_OUTPUT_DIR, 'log_etapa_12.txt')
os.makedirs(STAGE_12_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM (H2O COM CROSS-VALIDATION) DA ETAPA 12 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*70 + "\n\n")

print(f"Diretório de saída da Etapa 12 criado em: {STAGE_12_OUTPUT_DIR}")
h2o.init()

## 12.4. Função de Modelagem com Validação Cruzada

A função de treinamento do AutoML foi adaptada para refletir a nova estratégia. A principal mudança está na configuração do `H2OAutoML` com o parâmetro `nfolds=5` e na forma como os dados são divididos, criando um conjunto de treinamento maior para a CV e mantendo o conjunto de teste isolado.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def treinar_com_automl_cv(df, nome_modelo):
    """
    Executa o pipeline completo de treinamento com H2O AutoML e Validação Cruzada.
    """
    log_mensagem(f"--- INICIANDO TREINAMENTO H2O AutoML com CV PARA O MODELO: {nome_modelo} ---")

    TARGET = 'base_calculo_corrigida_2024'
    leaky_features = ['preco_m2_construido_corrigido_2024', 'preco_m2_terreno_corrigido_2024']
    features_cols = [col for col in df.columns if col != TARGET and col not in leaky_features]

    hf = h2o.H2OFrame(df[features_cols + [TARGET]])
    log_mensagem("DataFrame convertido para H2OFrame.")

    log_mensagem("Realizando divisão temporal: Treino (2006-2020) para CV, Teste (2021-2024) para avaliação final.")
    train_full = hf[hf['ANO'] <= 2020, :]
    test = hf[hf['ANO'] >= 2021, :]
    log_mensagem(f"Tamanhos dos conjuntos: Treino (para CV)={train_full.shape[0]}, Teste={test.shape[0]}")

    x = [col for col in features_cols if col != 'ANO']
    y = TARGET

    log_mensagem("Iniciando H2O AutoML com 5-fold Cross-Validation... (max_runtime_secs=600)")
    aml = H2OAutoML(
        max_runtime_secs=600,
        nfolds=5,
        seed=42,
        sort_metric="RMSE"
    )
    aml.train(x=x, y=y, training_frame=train_full, leaderboard_frame=test)

    lb = aml.leaderboard
    log_mensagem(f"\n--- Leaderboard (baseado em CV) para o Modelo {nome_modelo} ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        log_mensagem(lb.as_data_frame().to_string())

    leader_model = aml.leader
    log_mensagem(f"\nMelhor modelo encontrado (com base na CV): {leader_model.model_id}")

    perf_test = leader_model.model_performance(test_data=test)
    log_mensagem(f"\n--- Performance Final do Melhor Modelo no Conjunto de Teste ({nome_modelo}) ---")
    log_mensagem(str(perf_test))

    log_mensagem(f"--- FINALIZADO TREINAMENTO H2O COM CV PARA O MODELO: {nome_modelo} ---\n")

## 12.5. Execução da Pipeline de Modelagem Robusta

A célula final orquestra a execução da nova pipeline de modelagem. Ela carrega os datasets da Etapa 6 (que usam apenas `Latitude` e `Longitude`) e aplica a função de treinamento com Validação Cruzada para cada segmento de imóvel.

In [ ]:
log_mensagem("--- INICIANDO PROCESSAMENTO H2O COM CROSS-VALIDATION (ETAPA 12) ---")

try:
    df_vertical = pd.read_parquet(MODELING_DATASET_VERTICAL_PATH)
    treinar_com_automl_cv(df_vertical, "Vertical")
except Exception as e:
    log_mensagem(f"Erro ao processar dataset vertical com H2O e CV: {e}", tipo="ERRO CRÍTICO")

try:
    df_horizontal = pd.read_parquet(MODELING_DATASET_HORIZONTAL_PATH)
    treinar_com_automl_cv(df_horizontal, "Horizontal")
except Exception as e:
    log_mensagem(f"Erro ao processar dataset horizontal com H2O e CV: {e}", tipo="ERRO CRÍTICO")

log_mensagem("--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM (H2O COM CV) DA ETAPA 12 ---", tipo="SUMÁRIO")
h2o.cluster().shutdown()
log_mensagem("Cluster H2O desligado.")

# Etapa 13: Modelagem Final com H2O AutoML, CV e Features Híbridas

## 13.1. Introdução

Esta é a etapa culminante de nossa experimentação. Tendo estabelecido que a Validação Cruzada (CV) é a metodologia de treinamento mais robusta (Etapa 12) e que as features híbridas (`Latitude`/`Longitude` + `Cluster`) apresentam o maior potencial preditivo (Etapa 9.3), esta etapa une as duas abordagens.

O objetivo é treinar os modelos de H2O AutoML utilizando a Validação Cruzada no nosso dataset mais rico, que contém tanto a localização granular quanto a informação de zona. A hipótese é que esta combinação resultará nos modelos de mais alta performance de todo o projeto, representando o estado da arte para este problema com os dados disponíveis.

## 13.2. Objetivos

1.  **Combinar as Melhores Técnicas:** Unir a metodologia de treinamento com Validação Cruzada do H2O com o dataset que inclui a feature `localizacao_cluster`.
2.  **Treinar os Modelos Finais:** Executar a pipeline de modelagem final para os segmentos vertical e horizontal.
3.  **Obter a Performance Definitiva:** Avaliar os modelos resultantes no conjunto de teste para obter as métricas finais de performance do projeto.

In [ ]:
import os
import pandas as pd
from datetime import datetime
!pip install h2o -q
import h2o
from h2o.automl import H2OAutoML

DRIVE_BASE_PATH = '/content/drive/MyDrive/dados_tcc'

STAGE_09_1_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_09_1_kmeans_clustering')
CLUSTERED_DATASET_PATH = os.path.join(STAGE_09_1_OUTPUT_DIR, 'dataset_com_clusters.parquet')

STAGE_13_OUTPUT_DIR = os.path.join(DRIVE_BASE_PATH, 'stage_13_h2o_cv_hybrid')
LOG_FILE_PATH = os.path.join(STAGE_13_OUTPUT_DIR, 'log_etapa_13.txt')
os.makedirs(STAGE_13_OUTPUT_DIR, exist_ok=True)

with open(LOG_FILE_PATH, 'w') as f:
    f.write(f"--- LOG DE MODELAGEM FINAL (H2O + CV + HÍBRIDO) DA ETAPA 13 ---\n")
    f.write(f"Execução iniciada em: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write("="*70 + "\n\n")

print(f"Diretório de saída da Etapa 13 criado em: {STAGE_13_OUTPUT_DIR}")
h2o.init()

## 13.3. Funções de Preparação e Modelagem

Reutilizamos as funções de preparação de dados e adaptamos a função de modelagem do AutoML para garantir que ela utilize a Validação Cruzada e trate a feature `localizacao_cluster` como um fator categórico.

In [ ]:
def log_mensagem(mensagem, tipo='INFO'):
    """Registra uma mensagem formatada no console e no arquivo de log."""
    log_formatado = f"[{tipo}] {datetime.now().strftime('%H:%M:%S')} - {mensagem}"
    print(log_formatado)
    with open(LOG_FILE_PATH, 'a', encoding='utf-8') as f:
        f.write(log_formatado + "\n")

def preparar_dataset_para_modelagem(df):
    log_mensagem("Iniciando preparação completa do dataset clusterizado...")
    colunas_numericas = ['valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida','area_terreno', 'testada', 'fracao_ideal', 'area_construida']
    for col in colunas_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    df['valor_venal_referencia_proporcional'] = df['valor_venal_referencia'] * (df['proporcao_transmitida'] / 100)
    df['base_calculo'] = df[['valor_transacao', 'valor_venal_referencia_proporcional']].max(axis=1)
    inflacao_anual = {2006: 3.14, 2007: 4.46, 2008: 5.90, 2009: 4.31, 2010: 5.91, 2011: 6.50, 2012: 5.84, 2013: 5.91, 2014: 6.41, 2015: 10.67, 2016: 6.29, 2017: 2.95, 2018: 3.75, 2019: 4.31, 2020: 4.52, 2021: 10.06, 2022: 5.79, 2023: 4.62, 2024: 4.83}
    fatores_anual = {ano: 1 + (taxa / 100) for ano, taxa in inflacao_anual.items()}
    fatores_correcao = {}
    anos_ordenados = sorted(fatores_anual.keys(), reverse=True)
    fator_acumulado = 1.0
    for ano in anos_ordenados:
        fatores_correcao[ano] = fator_acumulado
        fator_acumulado *= fatores_anual[ano]
    df['fator_correcao'] = df['ANO'].map(fatores_correcao)
    df['base_calculo_corrigida_2024'] = (df['base_calculo'] * df['fator_correcao']).round(2)
    limite_inferior = df['base_calculo_corrigida_2024'].quantile(0.05)
    limite_superior = df['base_calculo_corrigida_2024'].quantile(0.95)
    df = df[(df['base_calculo_corrigida_2024'] >= limite_inferior) & (df['base_calculo_corrigida_2024'] <= limite_superior)].copy()
    colunas_para_remover = [
        'n_cadastro', 'matricula_imovel', 'cep', 'natureza_transacao', 'data_transacao', 'MES_ANO',
        'valor_financiado', 'tipo_financiamento', 'cartorio_de_registro', 'situacao_no_sql',
        'uso_iptu', 'padrao_iptu', 'descricao_uso_iptu', 'Endereço', 'Bairro', 'Cidade', 'Estado', 'Endereço completo',
        'valor_transacao', 'valor_venal_referencia', 'proporcao_transmitida',
        'valor_venal_referencia_proporcional', 'base_calculo_original', 'base_calculo', 'fator_correcao'
    ]
    df_final = df.drop(columns=[col for col in colunas_para_remover if col in df.columns])
    log_mensagem("Preparação completa do dataset concluída. Shape final: " + str(df_final.shape))
    return df_final

def treinar_com_automl_cv_hibrido(df, nome_modelo):
    """
    Executa a pipeline final com H2O AutoML, CV e features híbridas.
    """
    log_mensagem(f"--- INICIANDO TREINAMENTO FINAL (H2O+CV+Híbrido) PARA O MODELO: {nome_modelo} ---")

    TARGET = 'base_calculo_corrigida_2024'
    features_cols = [col for col in df.columns if col != TARGET]

    hf = h2o.H2OFrame(df)
    hf['localizacao_cluster'] = hf['localizacao_cluster'].asfactor()
    log_mensagem("DataFrame convertido para H2OFrame e 'localizacao_cluster' definido como fator.")

    train_full = hf[hf['ANO'] <= 2020, :]
    test = hf[hf['ANO'] >= 2021, :]
    log_mensagem(f"Tamanhos: Treino (para CV)={train_full.shape[0]}, Teste={test.shape[0]}")

    x = [col for col in features_cols if col != 'ANO']
    y = TARGET

    log_mensagem("Iniciando H2O AutoML com 5-fold Cross-Validation... (max_runtime_secs=600)")
    aml = H2OAutoML(max_runtime_secs=600, nfolds=5, seed=42, sort_metric="RMSE")
    aml.train(x=x, y=y, training_frame=train_full, leaderboard_frame=test)

    lb = aml.leaderboard
    log_mensagem(f"\n--- Leaderboard (CV) para o Modelo {nome_modelo} ---")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        log_mensagem(lb.as_data_frame().to_string())

    leader_model = aml.leader
    log_mensagem(f"\nMelhor modelo encontrado (com base na CV): {leader_model.model_id}")

    perf_test = leader_model.model_performance(test_data=test)
    log_mensagem(f"\n--- Performance Final do Melhor Modelo no Conjunto de Teste ({nome_modelo}) ---")
    log_mensagem(str(perf_test))

    log_mensagem(f"--- FINALIZADO TREINAMENTO FINAL PARA O MODELO: {nome_modelo} ---\n")

In [ ]:
log_mensagem("--- INICIANDO PROCESSAMENTO FINAL (H2O + CV + HÍBRIDO - ETAPA 13) ---")

try:
    df_clusterizado = pd.read_parquet(CLUSTERED_DATASET_PATH)
    log_mensagem(f"Dataset clusterizado da Etapa 9.1 carregado com {len(df_clusterizado)} registros.")

    df_modelagem = preparar_dataset_para_modelagem(df_clusterizado)

    df_modelagem['descricao_padrao_iptu'] = df_modelagem['descricao_padrao_iptu'].str.strip()
    df_vertical = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL VERTICAL'].drop(columns=['descricao_padrao_iptu', 'area_terreno', 'testada'])
    df_horizontal = df_modelagem[df_modelagem['descricao_padrao_iptu'] == 'RESIDENCIAL HORIZONTAL'].drop(columns=['descricao_padrao_iptu', 'fracao_ideal'])

    treinar_com_automl_cv_hibrido(df_vertical, "Vertical")
    treinar_com_automl_cv_hibrido(df_horizontal, "Horizontal")

except Exception as e:
    log_mensagem(f"Ocorreu um erro inesperado na Etapa 13: {e}", tipo='ERRO CRÍTICO')

log_mensagem("--- FINALIZAÇÃO DA PIPELINE DE MODELAGEM FINAL (ETAPA 13) ---", tipo="SUMÁRIO")
h2o.cluster().shutdown()
log_mensagem("Cluster H2O desligado.")